# Diclaimer

An atom BF based attack on BF segments

Original author: Peter Christen

Contact: peter.christen@anu.edu.au

School of Computing, The Australian National University, Canberra, ACT, 2600

Python 3 migration: Thiago Nóbrega <<thiagonobrega@gmail.com>>

---
Copyright 2021 Australian National University and others.
All Rights reserved.

---

This program is free software: you can redistribute it and/or modify it under
the terms of the GNU General Public License as published by the Free Software
Foundation, either version 3 of the License, or (at your option) any later
version.

This program is distributed in the hope that it will be useful, but WITHOUT
ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS
FOR A PARTICULAR PURPOSE. See the GNU General Public License for more
details.

You should have received a copy of the GNU General Public License along with
this program.  If not, see <http://www.gnu.org/licenses/>.

---

**Enviroment setup**
---

-   Install Libs
-   Load Dataset
-   Load libs

In [ ]:
!pip install numpy &> /dev/null
!pip install bitarray &> /dev/null
!pip install hashlib &> /dev/null
!pip install xxhash &> /dev/null

!rm /tmp/ncvoter* &> /dev/null
!rm /tmp/brpoli* &> /dev/null

!wget -P /tmp/ 'https://github.com/thiagonobrega/bcpprl-simplified/raw/master/dataset/ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz' &> /dev/null
!wget -P /tmp/ 'https://github.com/thiagonobrega/bcpprl-simplified/raw/master/dataset/ncvoter-20140619-temporal-balanced-ratio-1to1-b.csv.gz' &> /dev/null
!wget -P /tmp/ 'https://github.com/thiagonobrega/bcpprl-simplified/raw/master/dataset/sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b.csv.gz' &> /dev/null
!wget -P /tmp/ 'https://github.com/thiagonobrega/bcpprl-simplified/raw/master/dataset/ncvr-splits/sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split2.csv.gz' &> /dev/null
!wget -P /tmp/ 'https://github.com/thiagonobrega/bcpprl-simplified/raw/master/dataset/ncvr-splits/sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split3.csv.gz' &> /dev/null
!wget -P /tmp/ 'https://github.com/thiagonobrega/bcpprl-simplified/raw/master/dataset/ncvr-splits/sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split4.csv.gz' &> /dev/null
!wget -P /tmp/ 'https://github.com/thiagonobrega/bcpprl-simplified/raw/master/dataset/ncvr-splits/sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split5.csv.gz' &> /dev/null
!wget -P /tmp/ 'https://github.com/thiagonobrega/bcpprl-simplified/raw/master/dataset/ncvr-splits/sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split6.csv.gz' &> /dev/null

!wget -P /tmp/ 'https://github.com/thiagonobrega/bcpprl-simplified/raw/master/dataset/brpoliticians-federal-2014-all-1to1-a.csv.gz' &> /dev/null
!wget -P /tmp/ 'https://github.com/thiagonobrega/bcpprl-simplified/raw/master/dataset/brpoliticians-federal-2018-all-1to1-b.csv.gz' &> /dev/null
!wget -P /tmp/ 'https://github.com/thiagonobrega/bcpprl-simplified/raw/master/dataset/brpoliticians-federal-2018-5p-i2.csv.gz' &> /dev/null
!wget -P /tmp/ 'https://github.com/thiagonobrega/bcpprl-simplified/raw/master/dataset/brpoliticians-federal-2018-5p-i3.csv.gz' &> /dev/null
!wget -P /tmp/ 'https://github.com/thiagonobrega/bcpprl-simplified/raw/master/dataset/brpoliticians-federal-2018-5p-i4.csv.gz' &> /dev/null
!wget -P /tmp/ 'https://github.com/thiagonobrega/bcpprl-simplified/raw/master/dataset/brpoliticians-federal-2018-5p-i5.csv.gz' &> /dev/null
!wget -P /tmp/ 'https://github.com/thiagonobrega/bcpprl-simplified/raw/master/dataset/brpoliticians-federal-2018-5p-i6.csv.gz' &> /dev/null


import csv
import gzip
import hashlib
import math
import random
import os
import sys
import time

import bitarray
import numpy
import xxhash
import pandas as pd

Configure google drive to save the output

In [ ]:
from google.colab import drive
drive.mount("/gdrive/")
RESULTS_PATH='/gdrive/My Drive/Colab Notebooks/bc-pprlSegmentAttack/results/'

## Metodos

In [ ]:
def load_data_set_extract_q_grams(file_name, rec_id_col, use_attr_list,
                                  col_sep_char, header_line, q):
  """Load the given file, extract selected attributes, and convert each unique
     attribute value into a q-gram set.

     Return two dictionaries, one with attribute values as keys and their
     q-gram set as values. The second dictionary has q-grams as keys and for
     each a set of all the plain-text values that contain this q-gram.

     The function also returns the average number of q-grams per attribute
     value, and the set of all unique q-grams that occurred in any values.
     Also returns a string with the names of the encoded attributes.
  """

  start_time = time.time()

  
  if (file_name.endswith('gz')):
    #modified
    f = gzip.open(DATASET_PATH+file_name,mode='rt',errors="ignore")
  else:
    f = open(DATASET_PATH+file_name)

  csv_reader = csv.reader(f, delimiter=col_sep_char)

  print('Load data set from file:', file_name)
  print('  Attribute separator: %c' % (col_sep_char))
  if (header_line == True):
    # modificado
    # header_list = csv_reader.next()
    header_list = next(csv_reader)
    print('  Header line:', header_list)

  use_attr_name_list = []

  if (header_line == False):
    print('  Record identifier attribute number:', rec_id_col)
  else:
    print('  Record identifier attribute:', header_list[rec_id_col])
    print('    Attributes to use:')
    for attr_num in use_attr_list:
      use_attr_name = header_list[attr_num]
      print(use_attr_name)
      use_attr_name_list.append(use_attr_name)
    print('')
  print('')
  print('  Extract q-grams with q=%d' % (q))
  print('')

  encoded_attr_name_str = ';'.join(use_attr_name_list)

  qm1 = q-1  # Shorthand

  attr_val_q_gram_dict = {}
  q_gram_attr_val_dict = {}
  all_q_gram_set =       set()

  num_q_gram_per_attr_val_list = []  # To calculate statistics

  rec_num = 0

  for rec_list in csv_reader:
    rec_num += 1

    if (rec_num % 100000 == 0):
      time_used = time.time() - start_time
      print('  Processed %d records in %.3f sec (%.3f msec average)' % \
            (rec_num, time_used, 1000.0*time_used/rec_num))

    attr_val_list = []  # The attribute values to be converted into q-grams

    for attr_num in use_attr_list:
      attr_val_list.append(rec_list[attr_num].strip().lower())

    # Concatenate with whitespaces and keep as one string per record
    #
    attr_val = ' '.join(attr_val_list).strip()

    # Encode each attribute value once
    #
    if (attr_val not in attr_val_q_gram_dict):

      attr_q_gram_set = set([attr_val[i:i+q] for i in range(len(attr_val)-qm1)])

      if (len(attr_q_gram_set) > 0):

        # Keep the q-gram set for this attribute value
        #
        attr_val_q_gram_dict[attr_val] = attr_q_gram_set
        num_q_gram_per_attr_val_list.append(len(attr_q_gram_set))

        # Keep the attribute value for each of its q-grams
        #
        for q_gram in attr_q_gram_set:
          q_gram_attr_val_set = q_gram_attr_val_dict.get(q_gram, set())
          q_gram_attr_val_set.add(attr_val)
          q_gram_attr_val_dict[q_gram] = q_gram_attr_val_set

          all_q_gram_set.add(q_gram)  # And keep all unique q-grams

  time_used = time.time() - start_time
  print('  Processed %d records in %.2f sec (%.2f msec average)' % \
        (rec_num, time_used, 1000.0*time_used/rec_num))

  num_attr_val_q_gram_list = []
  
  #modified to python3
  for attr_val_set in q_gram_attr_val_dict.items():
  # for attr_val_set in q_gram_attr_val_dict.itervalues():
    num_attr_val_q_gram_list.append(len(attr_val_set))

  # print
  print('  Found %d unique attribute values' % (len(attr_val_q_gram_dict)))
  print('')
  print('  Number of attribute values per q-gram:')
  print('    Minimum: %d' % (min(num_attr_val_q_gram_list)));
  print('    Average: %.1f' % (numpy.mean(num_attr_val_q_gram_list)))
  print('    Median:  %d' % (numpy.median(num_attr_val_q_gram_list)))
  print('    Maximum: %d' % (max(num_attr_val_q_gram_list)))
  
  avr_num_q_gram = numpy.mean(num_q_gram_per_attr_val_list)

  print('  Average number of q-grams per attribte value: %.2f' % \
        (avr_num_q_gram))
  print('    Minimum and maximum number of q-grams: %d / %d' % \
        (min(num_q_gram_per_attr_val_list), max(num_q_gram_per_attr_val_list)))
  print('  Total number of unique q-grams in all values: %d' % \
        (len(all_q_gram_set)))
  print

  return attr_val_q_gram_dict, q_gram_attr_val_dict, avr_num_q_gram, \
         all_q_gram_set, encoded_attr_name_str

def gen_bloom_filter_dict(attr_val_q_gram_dict, hash_type, bf_len,
                          num_hash_funct):
  """Encode the q-gram sets in the given dictionary into Bloom filters of the
     given length using the given number of hash functions.

     Return a dictionary with keys being q-gram sets (as tuples) and values
     being Bloom filters (bit arrays), a dictionary of bit positions (keys)
     and which q-grams were hashed to then (values), and a dictionary of atom
     BFs which encode a single q-gram (one such BF for each unique q-gram).
  """

  start_time = time.time()

  print('Generate Bloom filter bit-patterns for %d q-gram sets' % \
        (len(attr_val_q_gram_dict)))
  print('  Bloom filter length:          ', bf_len)
  print('  Number of hash functions used:', num_hash_funct)
  print('  Hashing type used:            ', \
        {'dh':'Double hashing', 'rh':'Random hashing', 
         'xrh':'xHash Random hashing', 'xbf': 'xHash Xor Folding'}[hash_type])
  print('')

  bf_dict = {}  # One BF per q-gram set

  bit_pos_q_gram_dict = {}  # Bit pos. are keys, q-grams hashed to them values

  atom_bf_dict = {}  # Keys are q-grams, values their atom BF
  
  num_q_gram_set = 0  # Count how many q-gram sets processed

  bf_len_m1 = bf_len-1

  for (attr_val, q_gram_set) in attr_val_q_gram_dict.items():
    num_q_gram_set += 1

    if (num_q_gram_set % 10000 == 0):
      time_used = time.time() - start_time
      print('  Generated %d Bloom filters in %d sec (%.3f msec average)' % \
            (num_q_gram_set, time_used, 1000.0*time_used/num_q_gram_set))

    rec_bf = bitarray.bitarray(bf_len)
    rec_bf.setall(0)

    for q_gram in q_gram_set:  # Hash all q-grams into bits in the BF

      if (q_gram not in atom_bf_dict):  # We need to generate the atom BF
        atom_bf = bitarray.bitarray(bf_len)
        atom_bf.setall(0)

      
      ###
      ### Parte importante para o algoritmo utiliza apenas duas funções como 
      ### como hash, ou seja apenas duas funções...
      ###
      if (hash_type == 'dh'):  # Double hashing
        hex_str1 = BF_HASH_FUNCT1(q_gram).hexdigest()
        int1 =     int(hex_str1, 16)
        hex_str2 = BF_HASH_FUNCT2(q_gram).hexdigest()
        int2 =     int(hex_str2, 16)

        for i in range(1,num_hash_funct+1):  # Ensure i is not 0
          pos = int((int1 + i*int2) % bf_len)
          rec_bf[pos] = 1

          if (q_gram not in atom_bf_dict):
            atom_bf[pos] = 1

          bit_pos_q_gram_set = bit_pos_q_gram_dict.get(pos, set())
          bit_pos_q_gram_set.add(q_gram)
          bit_pos_q_gram_dict[pos] = bit_pos_q_gram_set

      ###
      ### o utilizado
      ###
      elif (hash_type == 'rh'):  # Random hashing
        random_seed = random.seed(q_gram)

        for i in range(num_hash_funct):
          pos = random.randint(0, bf_len_m1)
          rec_bf[pos] = 1

          if (q_gram not in atom_bf_dict):
            atom_bf[pos] = 1

          bit_pos_q_gram_set = bit_pos_q_gram_dict.get(pos, set())
          bit_pos_q_gram_set.add(q_gram)
          bit_pos_q_gram_dict[pos] = bit_pos_q_gram_set

      ###
      ### Funcao hash que utilizei
      ###
      elif (hash_type == 'xrh'):  # Xhash Random hashing
        for seed in range(num_hash_funct):
          pos=xxhash.xxh64(q_gram, seed=seed).intdigest() % bf_len
          rec_bf[pos] = 1
        
          if (q_gram not in atom_bf_dict):
            atom_bf[pos] = 1
          
          bit_pos_q_gram_set = bit_pos_q_gram_dict.get(pos, set())
          bit_pos_q_gram_set.add(q_gram)
          bit_pos_q_gram_dict[pos] = bit_pos_q_gram_set

      ###
      ### XOR FOlding

      elif (hash_type == 'xbf'):  # Xhash Random hashing
        for seed in range(num_hash_funct):
          
          fold_location = int(round( bf_len / 2 )) # xor pos
          pos=xxhash.xxh64(q_gram, seed=seed).intdigest() % fold_location
          rec_bf[pos] = 1
        
          if (q_gram not in atom_bf_dict):
            atom_bf[pos] = 1
          
          bit_pos_q_gram_set = bit_pos_q_gram_dict.get(pos, set())
          bit_pos_q_gram_set.add(q_gram)
          bit_pos_q_gram_dict[pos] = bit_pos_q_gram_set

      else:  # Should not happend
        raise Exception(hash_type)

      if (q_gram not in atom_bf_dict):
        atom_bf_dict[q_gram] = atom_bf

    if (hash_type == 'xbf'):
      fold_location = int(round( bf_len / 2 )) # xor pos
      bf_dict[attr_val] = rec_bf[0:fold_location]
    else:
      bf_dict[attr_val] = rec_bf


  num_q_gram_at_pos_list = []  # To calculate statistics

  for pos in sorted(bit_pos_q_gram_dict.keys()):
    num_q_gram_at_pos_list.append(len(bit_pos_q_gram_dict[pos]))

  print('  Number of q-grams assigned to bit positions:')
  print('    Minimum: %d' % (min(num_q_gram_at_pos_list)))
  print('    Average: %.1f' % (numpy.mean(num_q_gram_at_pos_list)))
  print('    Median:  %d' % (numpy.median(num_q_gram_at_pos_list)))
  print('    Maximum: %d' % (max(num_q_gram_at_pos_list)))
  print('  Generated atom BFs for %d q-grams' % (len(atom_bf_dict)))
  print('')

  assert len(bf_dict) == len(attr_val_q_gram_dict)

  return bf_dict, bit_pos_q_gram_dict, atom_bf_dict

######
#####
####
### SBF adptation
##
#

def get_bf_segments(bf_dict, bf_seg_len):
  """From the given Bloom filter dictionary, extract for each BF the segment
     of the required length.

     Return a new dictionary with the same keys and values being the BF
     segments.
  """

  bf_seg_dict = {}

  for (key_val, bf) in bf_dict.items():
    bf_seg_dict[key_val] = bf[:bf_seg_len]
    assert len(bf_seg_dict[key_val]) == bf_seg_len

  print('Extracted BF segments of length %d bits from %d BFs' % \
        (bf_seg_len, len(bf_seg_dict)))
  print

  return bf_seg_dict

def bf_segment_get_num_q_gram(bit_pos_q_gram_dict, bf_seg_len, num_hash_funct):
  """Get the number of unique q-grams that have been hashed into at least one
     bit position in the given BF segment. For each of these q-grams count in
     how many bit positions the q-gram is encoded.

     Returns the number of q-grams hashed into bits in the segment, the
     minimum, average, median, and maximum number of times a q-gram is hashed
     into a bit in the segment.
  """

  print('Get the number of unique q-grams that have been hashed into at ' + \
        'least one bit position in the given BF segment.')
  print('  Length of BF segment:', bf_seg_len)


  seg_q_gram_count_dict = {}  # Number of positions each q-gram is encoded into
  all_q_gram_count_dict = {}  # Should be number of hash functions for this

  for (pos, pos_q_gram_set) in bit_pos_q_gram_dict.items():
    for q_gram in pos_q_gram_set:
      all_q_gram_count_dict[q_gram] = all_q_gram_count_dict.get(q_gram,0) + 1
      if (pos < bf_seg_len):
        seg_q_gram_count_dict[q_gram] = seg_q_gram_count_dict.get(q_gram,0) + 1

  seg_q_gram_count_list = list(seg_q_gram_count_dict.values())
  all_q_gram_count_list = list(all_q_gram_count_dict.values())

  # Check for all that no count is larger than number of hash functions used
  #
  assert max(all_q_gram_count_list) <= num_hash_funct

  print('  There are %d unique q-grams encoded into bit positions in this ' \
        % (len(seg_q_gram_count_dict)) + 'segment, from a total of %d ' % \
          (len(all_q_gram_count_dict))+'q-grams encoded')
  print('    Number of positions each q-gram is hashed into in segment and ' + \
        'full BF:')
  print('      (note for the full BF this is likely less than the number of' + \
        ' hash functions (%d) due to collisions)' % (num_hash_funct))
  print('    Minimum: %d / %d' % (min(seg_q_gram_count_list),
                                  min(all_q_gram_count_list)))
  print('    Average: %.1f / %.1f' % (numpy.mean(seg_q_gram_count_list),
                                      numpy.mean(all_q_gram_count_list)))
  print('    Median:  %d / %d' % (numpy.median(seg_q_gram_count_list),
                                  numpy.median(all_q_gram_count_list)))
  print('    Maximum: %d / %d' % (max(seg_q_gram_count_list),
                                  max(all_q_gram_count_list)))
  print('')

  return len(seg_q_gram_count_dict), min(seg_q_gram_count_list), \
         numpy.mean(seg_q_gram_count_list), \
         numpy.median(seg_q_gram_count_list), max(seg_q_gram_count_list)

def bf_segment_atom_attack(ot_bf_seg_dict, my_atom_bf_seg_dict,
                           my_bit_pos_q_gram_dict, q_gram_attr_val_dict,
                           num_bf_to_attack):
  """Use atom BFs (for individual q-grams) to identify which q-grams can be
     encoded in a BF segment of the other DO. Then further filter possible
     q-grams by removing those known to be hashed to a bit position where a
     BF segment has a 0-bit (these could be false positive atoms due to
     collisions). Finally for each BF segment find the attribute value(s)
     that contain(s) the' largest number of the identified q-grams.

     Returns the number of correct 1-1, correct 1-many, wrong, and no matches
     where the sum of these numbers will be equal to 'num_bf_to_attack' (or
     equal to all BFs in the given BF dictionary if 'num_bf_to_attack' is
     given as 'all')
  """

  start_time = time.time()

  ot_num_bf_seg = len(ot_bf_seg_dict)

  if (num_bf_to_attack == 'all'):
    ot_num_bf_to_attack = ot_num_bf_seg
  else:
    assert num_bf_to_attack <= ot_num_bf_seg
    ot_num_bf_to_attack = num_bf_to_attack

  ot_attr_val_to_attack_list = random.sample(ot_bf_seg_dict.keys(),
                                             ot_num_bf_to_attack)

  print('Attack BF segments by atom BF matching BF segment bit patterns')
  print('  Number of other DO BF segments:   ', ot_num_bf_seg)
  print('    Number of BF segments to attack:', ot_num_bf_to_attack)
  print('  Number of atom BFs:               ', len(my_atom_bf_seg_dict))
  print('')

  num_bf = 0  # Count how many BFs processed

  num_corr_1_1_attr_matches = 0  # Counts of how good the attack is
  num_corr_1_m_attr_matches = 0
  num_wrong_1_matches =       0
  num_wrong_m_matches =       0
  num_no_matches =            0
  records_matched =           [] # the reidentified entities

  # How many more q-grams identified versus how many are encoded
  #
  diff_true_poss_size = []

  # For the second (filtering) step, count total reduction in possible q-grams
  #
  zero_bit_num_q_gram_removed = 0

  # Loop over all other BF segments and find the possible q-grams that could
  # have been encoded into a segment
  #
  for ot_attr_val in ot_attr_val_to_attack_list:
    ot_bf_seg = ot_bf_seg_dict[ot_attr_val]
    ot_true_q_gram_set = ot_attr_val_q_gram_dict[ot_attr_val] # True q-gram set

    num_bf += 1

    if (num_bf % 1000 == 0):
      time_used = time.time() - start_time
      print('  Analysed %d Bloom filters in %d sec (%.3f msec average)' % \
            (num_bf, time_used, 1000.0*time_used/num_bf))

    poss_q_gram_set = set()  # The q-grams possibly encoded in the BF segment

    # Step 1: The set of possibly encoded q-grams based on atoms that have all
    # their 1-bits set in the BF segment
    #
    for (atom_q_gram, atom_bf_seg) in my_atom_bf_seg_dict.items():
      if (atom_bf_seg & ot_bf_seg == atom_bf_seg):
        poss_q_gram_set.add(atom_q_gram)

    # Step 2: For 0-bits in the BF segment, remove from the set of possible
    # q-grams those that we know have been hashed to that position (these
    # possible q-grams could have been identified as atoms due to collisions)
    #
    for (pos, bit) in enumerate(ot_bf_seg):
      if (bit == False):  # A 0-bit

        # Get the q-grams we know have been hashed to this position
        #
        my_pos_q_gram_set = my_bit_pos_q_gram_dict[pos]

        # Remove them from the set of possible q-grams
        #
        for q_gram in my_pos_q_gram_set:
          if (q_gram in poss_q_gram_set):
            poss_q_gram_set.remove(q_gram)
            zero_bit_num_q_gram_removed += 1
            print('removed', q_gram)

    # Step 3: Get possible attribute values that contain one or more of the
    # possible q-grams, and select the attribute value with the largest number
    # of such q-grams to be the most likely value encoded in this BF
    #
    attr_val_accu = {}  # Keys will be attribute values, values counts of how
                        # many q-grams they have in the set of possible q-grams

    # TODO: this below process could be rather slow... need to improve *******

    max_count = -1

    for q_gram in poss_q_gram_set:
      q_gram_attr_val_set = q_gram_attr_val_dict[q_gram]
      for attr_val in q_gram_attr_val_set:
        count = attr_val_accu.get(attr_val, 0) + 1
        attr_val_accu[attr_val] = count
        max_count = max(count, max_count)

    found_attr_val_list = []

    # Find those attribute value(s) that have the maximum count (as how many
    # q-grams they have in the set of possible q-grams)
    #
    for (attr_val, count) in attr_val_accu.items():
      if (count == max_count):
        found_attr_val_list.append(attr_val)

#    print '  Most likely encoded attribute values(s):', found_attr_val_list
#    print '    True encoded attribute values:', ot_attr_val

    # Check if the found value(s) are correct:
    #
    if (len(found_attr_val_list) == 0):
      num_no_matches += 1
    elif (ot_attr_val in found_attr_val_list):
      if (len(found_attr_val_list) == 1):
        num_corr_1_1_attr_matches += 1
        records_matched.append(ot_attr_val) # reidentified entities list
      else:
        num_corr_1_m_attr_matches += 1
    else:
      if (len(found_attr_val_list) == 1):
        num_wrong_1_matches += 1
      else:
        num_wrong_m_matches += 1
  print('')

  print('  Filtering step using 0-bits removed a total %d q-grams' % \
        (zero_bit_num_q_gram_removed) + ' from sets of possible q-grams')
  print('')

  print('  Number of correct attribute values identified 1-to-1:    ' + \
        '%d (%.2f%%)' % (num_corr_1_1_attr_matches, 100.0 * \
                         num_corr_1_1_attr_matches/ot_num_bf_to_attack))
  print('  Number of correct attribute values identified 1-to-many: ' + \
        '%d (%.2f%%)' % (num_corr_1_m_attr_matches, 100.0 * \
                         num_corr_1_m_attr_matches/ot_num_bf_to_attack))
  print('  Number of wrong 1-to-1 attribute values identified:      ' + \
        '%d (%.2f%%)' % (num_wrong_1_matches, 100.0 * \
                         num_wrong_1_matches/ot_num_bf_to_attack))
  print('  Number of wrong 1-to-many attribute values identified:   ' + \
        '%d (%.2f%%)' % (num_wrong_m_matches, 100.0 * \
                         num_wrong_m_matches/ot_num_bf_to_attack))
  print('  Number of no attribute values identified:                ' + \
        '%d (%.2f%%)' % (num_no_matches, 100.0 * \
                         num_no_matches/ot_num_bf_to_attack))
  print('')

  assert (num_corr_1_1_attr_matches + num_corr_1_m_attr_matches + \
         num_wrong_1_matches + num_wrong_m_matches + num_no_matches) == \
         ot_num_bf_to_attack

  return (num_corr_1_1_attr_matches, num_corr_1_m_attr_matches, \
          num_wrong_1_matches, num_wrong_m_matches, num_no_matches , \
          records_matched)

def profile_reidentied_records(DATASET_PATH,my_data_set_name,ot_data_set_name,
                               col_sep_char,rec_id_col,attr_list):
  """This method uses the list of identified values to profile records
      in the original datasets.

     Returns the number of reidentified records (1:1), how many of these 
     are matches and nonmateches.
  """

  otds = pd.read_csv(DATASET_PATH+ot_data_set_name,sep=col_sep_char,
                     encoding = 'unicode_escape', engine ='python',
                     quoting=3)
  myds = pd.read_csv(DATASET_PATH+my_data_set_name,sep=col_sep_char,
                     encoding = 'unicode_escape', engine ='python',
                     quoting=3)

  otds.iloc[:,rec_id_col] = otds.iloc[:,rec_id_col].astype(str)
  myds.iloc[:,rec_id_col] = myds.iloc[:,rec_id_col].astype(str)

  otds['target'] = otds.iloc[:,attr_list].replace(numpy.nan, '', regex=True).agg(' '.join, axis=1)
  otds['target'] = otds["target"].str.lower()

  ot_ids_list = []
  multiple_ot_ids_list = []

  for record in records_matched:
    ot_id = list(otds[otds.target == record].iloc[:,rec_id_col].values)
    if len(ot_id) == 1:
      ot_ids_list.append(ot_id[0])
    else:
      ## two or more record are identical
      ## than we consider that for pratical terms the attack is unable to 
      ## reidentify properly
      multiple_ot_ids_list.append(ot_id)

  num_reidentified_ids = len(ot_ids_list)
  inter = set(ot_ids_list).intersection(set(myds.iloc[:,rec_id_col].values))
  num_reidentified_ids_matches = len(inter)
  num_reidentified_ids_no_matches = num_reidentified_ids - \
                                    num_reidentified_ids_matches

  return (num_reidentified_ids,num_reidentified_ids_matches,
          num_reidentified_ids_no_matches)

## Main

In [ ]:
#@title Run attack { form-width: "50%" }

# NUM_MATCHES_TO_TRY = 1000  # or 'all'

# initial parameters
BF_HASH_FUNCT1 = hashlib.sha1
BF_HASH_FUNCT2 = hashlib.md5
random.seed(42)

DATASET_PATH='/tmp/'
try:
  RESULTS_PATH
except NameError:
  RESULTS_PATH='/tmp/'

command_line_call = sys.argv


#@markdown -----------------
#@markdown ##### Dataset:
dataset = "ncvoter-sample" #@param ["ncvoter","ncvoter-sample","ncvoter-sample-nm-s2","ncvoter-sample-nm-s3","ncvoter-sample-nm-s4","ncvoter-sample-nm-s5","ncvoter-sample-nm-s6","politicians","politicians-split-2","politicians-split-3","politicians-split-4","politicians-split-5","politicians-split-6"]

if dataset == 'politicians':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-all-1to1-b.csv.gz'
if dataset == 'politicians-split-2':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i2.csv.gz'
if dataset == 'politicians-split-3':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i3.csv.gz'
if dataset == 'politicians-split-4':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i4.csv.gz'
if dataset == 'politicians-split-5':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i5.csv.gz'
if dataset == 'politicians-split-6':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i6.csv.gz'

if dataset == 'ncvoter':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-b.csv.gz'
if dataset == 'ncvoter-sample':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b.csv.gz'
if dataset == 'ncvoter-sample-nm-s2':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split2.csv.gz'
if dataset == 'ncvoter-sample-nm-s3':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split3.csv.gz'
if dataset == 'ncvoter-sample-nm-s4':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split4.csv.gz'
if dataset == 'ncvoter-sample-nm-s5':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split5.csv.gz'
if dataset == 'ncvoter-sample-nm-s6':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split6.csv.gz'

rec_id_col   = 0   #@param {type:"integer"}

#voter_reg_num,name_prefix,first_name,middle_name,last_name,
#name_suffix,age,gender,race,ethnic,
#street_address,city,state,zip_code,full_phone_num,
#birth_place,register_date,download_month
#@markdown ###### attr_list parameters
#@markdown | ATTRS | POS | ATTRS | POS |
#@markdown |:---:|:---:|:---:|:---:|
#@markdown |<td colspan=2>NCVR <td colspan=2>BR
#@markdown |first_name |3|first_name |1|
#@markdown |first_name , last_name|3,5|first_name , last_name|1,2|
#@markdown | full_name |3,4,5| full_name |3|
#@markdown |first_name , last_name , address|3,5,11|full_name, address|3,9,10|
# <sup> NCVR </sup>
# > <sup>3,5 - first_name , last_name</sup>\
# > <sup>3,5,11 - first_name , last_name, street_address</sup>\
# > <sup>3,5,6,7,12 - first_name , last_name, age, gender, city</sup>
attr_list        =    [3,5]#@param {type:"raw"}
#@markdown ###### col separator [(',' - ncvr),(';' - brpol)]
col_sep_char = ',' #@param {type:"string"}[',',';']
header_line_flag = True #@param {type:"raw"}

#@markdown -----------------
#@markdown #### Anonymization Parameter:

q =  2#@param {type:"integer"}
hash_type = "xrh" #@param ["rh","xrh","xbf","dh"]
num_hash_funct = "opt" #@param ["opt", "opt_half", "opt_quarter"]
bf_len =  200#@param {type:"integer"}

#ajustar depois
bf_seg_perc_list = [5] #@param {type:"raw"}
num_bf_to_attack =  'all'#@param {type:"raw"}

main_start_time = time.time()

# command_line_call = sys.argv
# q =                int(sys.argv[1])
# hash_type =        sys.argv[2].lower()
# num_hash_funct =   sys.argv[3]
# bf_len =           int(sys.argv[4])
# my_data_set_name = sys.argv[5]
# ot_data_set_name = sys.argv[6]
# rec_id_col =       int(sys.argv[7])
# col_sep_char =     sys.argv[8]
# header_line_flag = eval(sys.argv[9])
# attr_list =        eval(sys.argv[10])
# bf_seg_perc_list = eval(sys.argv[11])
# num_bf_to_attack = sys.argv[12]

assert q >= 1, q
assert hash_type in ['dh','rh','xrh','xbf'], hash_type
if num_hash_funct.isdigit():
  num_hash_funct = int(num_hash_funct)
  assert num_hash_funct >= 1, num_hash_funct
else:
  assert num_hash_funct in ['opt', 'opt_half', 'opt_quarter'], num_hash_funct
assert bf_len > 1, bf_len
#
assert rec_id_col >= 0, rec_id_col
assert header_line_flag in [True,False], header_line_flag
assert isinstance(attr_list, list), attr_list
assert isinstance(bf_seg_perc_list, list), bf_seg_perc_list
if (num_bf_to_attack != 'all'):
  num_bf_to_attack = int(num_bf_to_attack)
  assert num_bf_to_attack > 1, num_bf_to_attack

my_base_data_set_name = my_data_set_name.split('/')[-1].replace('.csv', '')
my_base_data_set_name = my_base_data_set_name.replace('.gz','')
ot_base_data_set_name = ot_data_set_name.split('/')[-1].replace('.csv', '')
ot_base_data_set_name = ot_base_data_set_name.replace('.gz','')

assert ',' not in my_base_data_set_name
assert ',' not in ot_base_data_set_name

print('')
print('-'*80)
print('')

ds_name = my_data_set_name.split('-')[0]
res_file = 'real-'+ds_name+'-'+str(num_bf_to_attack)+'-'+str(hash_type)+'-'+str(bf_len)+'.csv'
result_file_exists = os.path.exists(RESULTS_PATH + res_file)

f = open(RESULTS_PATH + res_file, "a")



# -----------------------------------------------------------------------------
# Step 1: Load the data sets and extract q-grams for selected attributes
#
my_attr_val_q_gram_dict, my_q_gram_attr_val_dict, my_avr_num_q_gram, \
   my_all_q_gram_set, my_encoded_attr_name_str = \
                       load_data_set_extract_q_grams(my_data_set_name,
                                                     rec_id_col, attr_list,
                                                     col_sep_char,
                                                     header_line_flag, q)

ot_attr_val_q_gram_dict, ot_q_gram_attr_val_dict, ot_avr_num_q_gram, \
   ot_all_q_gram_set, ot_encoded_attr_name_str = \
                       load_data_set_extract_q_grams(ot_data_set_name,
                                                     rec_id_col, attr_list,
                                                     col_sep_char,
                                                     header_line_flag, q)

assert my_encoded_attr_name_str == ot_encoded_attr_name_str

# Generate a dictionary with all q-grams and their attribute values from both
# data sets
#
my_num_q_gram = len(my_q_gram_attr_val_dict)
ot_num_q_gram = len(ot_q_gram_attr_val_dict)

all_q_gram_attr_val_dict =    {}

#modified to python3
for (q_gram, attr_val_set) in my_q_gram_attr_val_dict.items():
  all_q_gram_attr_val_dict[q_gram] = attr_val_set
for (q_gram, attr_val_set) in ot_q_gram_attr_val_dict.items():
  q_gram_attr_val_set = all_q_gram_attr_val_dict.get(q_gram, set())
  q_gram_attr_val_set = q_gram_attr_val_set | attr_val_set
  all_q_gram_attr_val_dict[q_gram] = q_gram_attr_val_set

num_all_q_gram =    len(all_q_gram_attr_val_dict)
num_common_q_gram = len(set(my_q_gram_attr_val_dict.keys()) & \
                        set(ot_q_gram_attr_val_dict.keys()))

print('A total of %d unique q-grams occur in both data set' % (num_all_q_gram))
print('  Of these, %d occur in both data sets' % (num_common_q_gram))
print('')

num_common_attr_val = len(set(my_attr_val_q_gram_dict.keys()) & \
                          set(ot_attr_val_q_gram_dict.keys()))
num_my_attr_val = len(my_attr_val_q_gram_dict)
num_ot_attr_val = len(ot_attr_val_q_gram_dict)

jacc_common_attr_val = float(num_common_attr_val) / \
  (num_my_attr_val + num_ot_attr_val - num_common_attr_val)

print('My database contains %d unique attribute values' % (num_my_attr_val))
print('The other database contains %d unique attribute values' % \
      (num_ot_attr_val))
print('  Number of unique attribute values in common: %d' % \
      (num_common_attr_val))
print('    Jaccard based overlap of common attribute values: %.1f%%' % \
      (100.0*jacc_common_attr_val))
print('')

# -----------------------------------------------------------------------------
# Step 2: Generate Bloom filters from q-gram sets
#
if (num_hash_funct in ['opt', 'opt_half', 'opt_quarter']):

  # Set number of hash functions to have in average 50% of bits set to 1
  # (Linking Sensitive Data book, 2020)
  # num_hash_funct = int(math.ceil(0.5 * BF_LEN / \
  #                                math.floor(avrg_num_q_gram)))
  #
  opt_num_hash_funct = int(round(numpy.log(2.0) * float(bf_len) / \
                                 my_avr_num_q_gram))
  if (num_hash_funct == 'opt'):
    num_hash_funct = opt_num_hash_funct
  elif (num_hash_funct == 'opt_half'):
    num_hash_funct = int(round(opt_num_hash_funct / 2.0))
  elif (num_hash_funct == 'opt_quarter'):
    num_hash_funct = int(round(opt_num_hash_funct / 4.0))
  else:
    raise Exception(num_hash_funct)

my_bf_dict, my_bit_pos_q_gram_dict, my_atom_bf_dict = \
    gen_bloom_filter_dict(my_attr_val_q_gram_dict, hash_type, bf_len,
                          num_hash_funct)


ot_bf_dict, ot_bit_pos_q_gram_dict, ot_atom_bf_dict = \
    gen_bloom_filter_dict(ot_attr_val_q_gram_dict, hash_type, bf_len,
                          num_hash_funct)

#rodando ate aqui

# Header string for results output
#
header = '### command_line_call, encoded_attr_name_str,num_bf_to_attack, ' +\
      'hash_type, num_hash_funct, num_my_attr_val, num_ot_attr_val, ' +\
      'num_common_attr_val, jacc_common_perc, my_num_q_gram, ot_num_q_gram, ' +\
      'num_all_q_gram, num_common_q_gram, bf_seg_len, bf_seg_perc, ' +\
      'my_num_q_gram_bf_seg, my_min_num_bit_pos, my_avr_num_bit_pos, ' +\
      'my_med_num_bit_pos, my_max_num_bit_pos, ot_num_q_gram_bf_seg, ' +\
      'ot_min_num_bit_pos, ot_avr_num_bit_pos, ot_med_num_bit_pos, ' +\
      'ot_max_num_bit_pos, atom_num_corr_1_1_attr_matches, ' +\
      'atom_num_corr_1_m_attr_matches, atom_num_wrong_1_matches, ' +\
      'atom_num_wrong_m_matches, atom_num_no_matches, num_reidentified_ids, ' +\
      'num_reidentified_ids_matches, num_reidentified_ids_no_matches, ' +\
      'reidentified_id_list'

print(header)

if result_file_exists == False:
  f.write(header.split('### ')[1]+'\n')

# -----------------------------------------------------------------------------
# Step 3: Run the segment attack with different segment percentages
#
for bf_seg_perc in bf_seg_perc_list:

  print('-'*80)
  print()

  # Generate a string to be printed as the result summary
  #
  res_str = '### "' + ' '.join(command_line_call)
  res_str = res_str.replace(' , ', ' comma ')  # The separator in input file
  res_str = res_str.replace(',', ';') + '", '

  # Details of how many attribute values in the two data sets, and how many
  # occur in common in both data sets
  #
  res_str += '%s, %s, %s, %d, %d, %d, %d, %.1f, ' % (my_encoded_attr_name_str,
                                                str(num_bf_to_attack),
                                                hash_type,
                                                num_hash_funct, num_my_attr_val,
                                                num_ot_attr_val,
                                                num_common_attr_val,
                                                100.0*jacc_common_attr_val)

  # Details of how many q-grams in the two data sets, how many in total,
  # and how many occur occur in common in both data sets
  #
  res_str += '%d, %d, %d, %d, ' % (my_num_q_gram, ot_num_q_gram, \
                                   num_all_q_gram, num_common_q_gram)

  if hash_type == 'xbf':
    bf_seg_len = int((float(bf_len)/2)*bf_seg_perc/100)
  else:
    bf_seg_len = int(float(bf_len)*bf_seg_perc/100)

  # Get the BF segments from the BFs of both data sets
  #
  my_bf_seg_dict = get_bf_segments(my_bf_dict, bf_seg_len)
  ot_bf_seg_dict = get_bf_segments(ot_bf_dict, bf_seg_len)
  
  my_atom_bf_seg_dict = get_bf_segments(my_atom_bf_dict, bf_seg_len)
  
  my_num_q_gram_bf_seg, my_min_num_bit_pos, my_avr_num_bit_pos, \
         my_med_num_bit_pos, my_max_num_bit_pos = \
                  bf_segment_get_num_q_gram(my_bit_pos_q_gram_dict,
                                            bf_seg_len, num_hash_funct)
  ot_num_q_gram_bf_seg, ot_min_num_bit_pos, ot_avr_num_bit_pos, \
         ot_med_num_bit_pos, ot_max_num_bit_pos = \
                  bf_segment_get_num_q_gram(ot_bit_pos_q_gram_dict,
                                            bf_seg_len, num_hash_funct)

  # Add BF segment information to result string
  #
  res_str += '%d, %d%%, %d, %d, %.2f, %d, %d, ' % (bf_seg_len, bf_seg_perc, \
                                                 my_num_q_gram_bf_seg, \
                                                 my_min_num_bit_pos, \
                                                 my_avr_num_bit_pos, \
                                                 my_med_num_bit_pos, \
                                                 my_max_num_bit_pos)
  res_str += '%d, %d, %.2f, %d, %d, ' % (ot_num_q_gram_bf_seg, \
                                       ot_min_num_bit_pos, \
                                       ot_avr_num_bit_pos, \
                                       ot_med_num_bit_pos, \
                                       ot_max_num_bit_pos)

  # Atom based attack
  #
  num_corr_1_1_attr_matches, num_corr_1_m_attr_matches, \
      num_wrong_1_matches, num_wrong_m_matches, \
      num_no_matches,records_matched = bf_segment_atom_attack(ot_bf_seg_dict,
                                              my_atom_bf_seg_dict,
                                              my_bit_pos_q_gram_dict,
                                              all_q_gram_attr_val_dict,
                                              num_bf_to_attack)

  res_str += '%d, %d, %d, %d, %d, ' % (num_corr_1_1_attr_matches, \
                                     num_corr_1_m_attr_matches, \
                                     num_wrong_1_matches, num_wrong_m_matches, \
                                     num_no_matches)

  # profile 
  #
  num_reidentified_ids,num_reidentified_ids_matches, \
  num_reidentified_ids_no_matches = profile_reidentied_records(DATASET_PATH,
                                          my_data_set_name,ot_data_set_name,
                                          col_sep_char,rec_id_col, attr_list)
  
  res_str += '%d, %d, %d,' % (num_reidentified_ids,num_reidentified_ids_matches,
                              num_reidentified_ids_no_matches)
  
  saida_lista = ''
  for r_id in records_matched:
    saida_lista += str(r_id)+'#'

  res_str += '%s' % (saida_lista) 
  # Print result line for CSV generation
  #

  print(res_str)
  f.write(res_str.split('### ')[1]+'\n') #savinf csv file
  f.flush()

f.close()


In [ ]:
f.close()

In [ ]:
#@title Run attack { form-width: "50%" }

# NUM_MATCHES_TO_TRY = 1000  # or 'all'

# initial parameters
BF_HASH_FUNCT1 = hashlib.sha1
BF_HASH_FUNCT2 = hashlib.md5
random.seed(42)

DATASET_PATH='/tmp/'
try:
  RESULTS_PATH
except NameError:
  RESULTS_PATH='/tmp/'

command_line_call = sys.argv


#@markdown -----------------
#@markdown ##### Dataset:
dataset = "ncvoter-sample-nm-s2" #@param ["ncvoter","ncvoter-sample","ncvoter-sample-nm-s2","ncvoter-sample-nm-s3","ncvoter-sample-nm-s4","ncvoter-sample-nm-s5","ncvoter-sample-nm-s6","politicians","politicians-split-2","politicians-split-3","politicians-split-4","politicians-split-5","politicians-split-6"]

if dataset == 'politicians':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-all-1to1-b.csv.gz'
if dataset == 'politicians-split-2':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i2.csv.gz'
if dataset == 'politicians-split-3':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i3.csv.gz'
if dataset == 'politicians-split-4':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i4.csv.gz'
if dataset == 'politicians-split-5':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i5.csv.gz'
if dataset == 'politicians-split-6':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i6.csv.gz'

if dataset == 'ncvoter':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-b.csv.gz'
if dataset == 'ncvoter-sample':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b.csv.gz'
if dataset == 'ncvoter-sample-nm-s2':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split2.csv.gz'
if dataset == 'ncvoter-sample-nm-s3':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split3.csv.gz'
if dataset == 'ncvoter-sample-nm-s4':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split4.csv.gz'
if dataset == 'ncvoter-sample-nm-s5':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split5.csv.gz'
if dataset == 'ncvoter-sample-nm-s6':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split6.csv.gz'

rec_id_col   = 0   #@param {type:"integer"}

#voter_reg_num,name_prefix,first_name,middle_name,last_name,
#name_suffix,age,gender,race,ethnic,
#street_address,city,state,zip_code,full_phone_num,
#birth_place,register_date,download_month
#@markdown ###### attr_list parameters
#@markdown | ATTRS | POS | ATTRS | POS |
#@markdown |:---:|:---:|:---:|:---:|
#@markdown |<td colspan=2>NCVR <td colspan=2>BR
#@markdown |first_name |3|first_name |1|
#@markdown |first_name , last_name|3,5|first_name , last_name|1,2|
#@markdown | full_name |3,4,5| full_name |3|
#@markdown |first_name , last_name , address|3,5,11|full_name, address|3,9,10|
# <sup> NCVR </sup>
# > <sup>3,5 - first_name , last_name</sup>\
# > <sup>3,5,11 - first_name , last_name, street_address</sup>\
# > <sup>3,5,6,7,12 - first_name , last_name, age, gender, city</sup>
attr_list        =    [3,5]#@param {type:"raw"}
#@markdown ###### col separator [(',' - ncvr),(';' - brpol)]
col_sep_char = ',' #@param {type:"string"}[',',';']
header_line_flag = True #@param {type:"raw"}

#@markdown -----------------
#@markdown #### Anonymization Parameter:

q =  2#@param {type:"integer"}
hash_type = "xrh" #@param ["rh","xrh","xbf","dh"]
num_hash_funct = "opt" #@param ["opt", "opt_half", "opt_quarter"]
bf_len =  200#@param {type:"integer"}

#ajustar depois
bf_seg_perc_list = [10] #@param {type:"raw"}
num_bf_to_attack =  'all'#@param {type:"raw"}

main_start_time = time.time()

# command_line_call = sys.argv
# q =                int(sys.argv[1])
# hash_type =        sys.argv[2].lower()
# num_hash_funct =   sys.argv[3]
# bf_len =           int(sys.argv[4])
# my_data_set_name = sys.argv[5]
# ot_data_set_name = sys.argv[6]
# rec_id_col =       int(sys.argv[7])
# col_sep_char =     sys.argv[8]
# header_line_flag = eval(sys.argv[9])
# attr_list =        eval(sys.argv[10])
# bf_seg_perc_list = eval(sys.argv[11])
# num_bf_to_attack = sys.argv[12]

assert q >= 1, q
assert hash_type in ['dh','rh','xrh','xbf'], hash_type
if num_hash_funct.isdigit():
  num_hash_funct = int(num_hash_funct)
  assert num_hash_funct >= 1, num_hash_funct
else:
  assert num_hash_funct in ['opt', 'opt_half', 'opt_quarter'], num_hash_funct
assert bf_len > 1, bf_len
#
assert rec_id_col >= 0, rec_id_col
assert header_line_flag in [True,False], header_line_flag
assert isinstance(attr_list, list), attr_list
assert isinstance(bf_seg_perc_list, list), bf_seg_perc_list
if (num_bf_to_attack != 'all'):
  num_bf_to_attack = int(num_bf_to_attack)
  assert num_bf_to_attack > 1, num_bf_to_attack

my_base_data_set_name = my_data_set_name.split('/')[-1].replace('.csv', '')
my_base_data_set_name = my_base_data_set_name.replace('.gz','')
ot_base_data_set_name = ot_data_set_name.split('/')[-1].replace('.csv', '')
ot_base_data_set_name = ot_base_data_set_name.replace('.gz','')

assert ',' not in my_base_data_set_name
assert ',' not in ot_base_data_set_name

print('')
print('-'*80)
print('')

ds_name = my_data_set_name.split('-')[0]
res_file = 'real-'+ds_name+'-'+str(num_bf_to_attack)+'-'+str(hash_type)+'-'+str(bf_len)+'.csv'
result_file_exists = os.path.exists(RESULTS_PATH + res_file)

f = open(RESULTS_PATH + res_file, "a")



# -----------------------------------------------------------------------------
# Step 1: Load the data sets and extract q-grams for selected attributes
#
my_attr_val_q_gram_dict, my_q_gram_attr_val_dict, my_avr_num_q_gram, \
   my_all_q_gram_set, my_encoded_attr_name_str = \
                       load_data_set_extract_q_grams(my_data_set_name,
                                                     rec_id_col, attr_list,
                                                     col_sep_char,
                                                     header_line_flag, q)

ot_attr_val_q_gram_dict, ot_q_gram_attr_val_dict, ot_avr_num_q_gram, \
   ot_all_q_gram_set, ot_encoded_attr_name_str = \
                       load_data_set_extract_q_grams(ot_data_set_name,
                                                     rec_id_col, attr_list,
                                                     col_sep_char,
                                                     header_line_flag, q)

assert my_encoded_attr_name_str == ot_encoded_attr_name_str

# Generate a dictionary with all q-grams and their attribute values from both
# data sets
#
my_num_q_gram = len(my_q_gram_attr_val_dict)
ot_num_q_gram = len(ot_q_gram_attr_val_dict)

all_q_gram_attr_val_dict =    {}

#modified to python3
for (q_gram, attr_val_set) in my_q_gram_attr_val_dict.items():
  all_q_gram_attr_val_dict[q_gram] = attr_val_set
for (q_gram, attr_val_set) in ot_q_gram_attr_val_dict.items():
  q_gram_attr_val_set = all_q_gram_attr_val_dict.get(q_gram, set())
  q_gram_attr_val_set = q_gram_attr_val_set | attr_val_set
  all_q_gram_attr_val_dict[q_gram] = q_gram_attr_val_set

num_all_q_gram =    len(all_q_gram_attr_val_dict)
num_common_q_gram = len(set(my_q_gram_attr_val_dict.keys()) & \
                        set(ot_q_gram_attr_val_dict.keys()))

print('A total of %d unique q-grams occur in both data set' % (num_all_q_gram))
print('  Of these, %d occur in both data sets' % (num_common_q_gram))
print('')

num_common_attr_val = len(set(my_attr_val_q_gram_dict.keys()) & \
                          set(ot_attr_val_q_gram_dict.keys()))
num_my_attr_val = len(my_attr_val_q_gram_dict)
num_ot_attr_val = len(ot_attr_val_q_gram_dict)

jacc_common_attr_val = float(num_common_attr_val) / \
  (num_my_attr_val + num_ot_attr_val - num_common_attr_val)

print('My database contains %d unique attribute values' % (num_my_attr_val))
print('The other database contains %d unique attribute values' % \
      (num_ot_attr_val))
print('  Number of unique attribute values in common: %d' % \
      (num_common_attr_val))
print('    Jaccard based overlap of common attribute values: %.1f%%' % \
      (100.0*jacc_common_attr_val))
print('')

# -----------------------------------------------------------------------------
# Step 2: Generate Bloom filters from q-gram sets
#
if (num_hash_funct in ['opt', 'opt_half', 'opt_quarter']):

  # Set number of hash functions to have in average 50% of bits set to 1
  # (Linking Sensitive Data book, 2020)
  # num_hash_funct = int(math.ceil(0.5 * BF_LEN / \
  #                                math.floor(avrg_num_q_gram)))
  #
  opt_num_hash_funct = int(round(numpy.log(2.0) * float(bf_len) / \
                                 my_avr_num_q_gram))
  if (num_hash_funct == 'opt'):
    num_hash_funct = opt_num_hash_funct
  elif (num_hash_funct == 'opt_half'):
    num_hash_funct = int(round(opt_num_hash_funct / 2.0))
  elif (num_hash_funct == 'opt_quarter'):
    num_hash_funct = int(round(opt_num_hash_funct / 4.0))
  else:
    raise Exception(num_hash_funct)

my_bf_dict, my_bit_pos_q_gram_dict, my_atom_bf_dict = \
    gen_bloom_filter_dict(my_attr_val_q_gram_dict, hash_type, bf_len,
                          num_hash_funct)


ot_bf_dict, ot_bit_pos_q_gram_dict, ot_atom_bf_dict = \
    gen_bloom_filter_dict(ot_attr_val_q_gram_dict, hash_type, bf_len,
                          num_hash_funct)

#rodando ate aqui

# Header string for results output
#
header = '### command_line_call, encoded_attr_name_str,num_bf_to_attack, ' +\
      'hash_type, num_hash_funct, num_my_attr_val, num_ot_attr_val, ' +\
      'num_common_attr_val, jacc_common_perc, my_num_q_gram, ot_num_q_gram, ' +\
      'num_all_q_gram, num_common_q_gram, bf_seg_len, bf_seg_perc, ' +\
      'my_num_q_gram_bf_seg, my_min_num_bit_pos, my_avr_num_bit_pos, ' +\
      'my_med_num_bit_pos, my_max_num_bit_pos, ot_num_q_gram_bf_seg, ' +\
      'ot_min_num_bit_pos, ot_avr_num_bit_pos, ot_med_num_bit_pos, ' +\
      'ot_max_num_bit_pos, atom_num_corr_1_1_attr_matches, ' +\
      'atom_num_corr_1_m_attr_matches, atom_num_wrong_1_matches, ' +\
      'atom_num_wrong_m_matches, atom_num_no_matches, num_reidentified_ids, ' +\
      'num_reidentified_ids_matches, num_reidentified_ids_no_matches, ' +\
      'reidentified_id_list'

print(header)

if result_file_exists == False:
  f.write(header.split('### ')[1]+'\n')

# -----------------------------------------------------------------------------
# Step 3: Run the segment attack with different segment percentages
#
for bf_seg_perc in bf_seg_perc_list:

  print('-'*80)
  print()

  # Generate a string to be printed as the result summary
  #
  res_str = '### "' + ' '.join(command_line_call)
  res_str = res_str.replace(' , ', ' comma ')  # The separator in input file
  res_str = res_str.replace(',', ';') + '", '

  # Details of how many attribute values in the two data sets, and how many
  # occur in common in both data sets
  #
  res_str += '%s, %s, %s, %d, %d, %d, %d, %.1f, ' % (my_encoded_attr_name_str,
                                                str(num_bf_to_attack),
                                                hash_type,
                                                num_hash_funct, num_my_attr_val,
                                                num_ot_attr_val,
                                                num_common_attr_val,
                                                100.0*jacc_common_attr_val)

  # Details of how many q-grams in the two data sets, how many in total,
  # and how many occur occur in common in both data sets
  #
  res_str += '%d, %d, %d, %d, ' % (my_num_q_gram, ot_num_q_gram, \
                                   num_all_q_gram, num_common_q_gram)

  if hash_type == 'xbf':
    bf_seg_len = int((float(bf_len)/2)*bf_seg_perc/100)
  else:
    bf_seg_len = int(float(bf_len)*bf_seg_perc/100)

  # Get the BF segments from the BFs of both data sets
  #
  my_bf_seg_dict = get_bf_segments(my_bf_dict, bf_seg_len)
  ot_bf_seg_dict = get_bf_segments(ot_bf_dict, bf_seg_len)
  
  my_atom_bf_seg_dict = get_bf_segments(my_atom_bf_dict, bf_seg_len)
  
  my_num_q_gram_bf_seg, my_min_num_bit_pos, my_avr_num_bit_pos, \
         my_med_num_bit_pos, my_max_num_bit_pos = \
                  bf_segment_get_num_q_gram(my_bit_pos_q_gram_dict,
                                            bf_seg_len, num_hash_funct)
  ot_num_q_gram_bf_seg, ot_min_num_bit_pos, ot_avr_num_bit_pos, \
         ot_med_num_bit_pos, ot_max_num_bit_pos = \
                  bf_segment_get_num_q_gram(ot_bit_pos_q_gram_dict,
                                            bf_seg_len, num_hash_funct)

  # Add BF segment information to result string
  #
  res_str += '%d, %d%%, %d, %d, %.2f, %d, %d, ' % (bf_seg_len, bf_seg_perc, \
                                                 my_num_q_gram_bf_seg, \
                                                 my_min_num_bit_pos, \
                                                 my_avr_num_bit_pos, \
                                                 my_med_num_bit_pos, \
                                                 my_max_num_bit_pos)
  res_str += '%d, %d, %.2f, %d, %d, ' % (ot_num_q_gram_bf_seg, \
                                       ot_min_num_bit_pos, \
                                       ot_avr_num_bit_pos, \
                                       ot_med_num_bit_pos, \
                                       ot_max_num_bit_pos)

  # Atom based attack
  #
  num_corr_1_1_attr_matches, num_corr_1_m_attr_matches, \
      num_wrong_1_matches, num_wrong_m_matches, \
      num_no_matches,records_matched = bf_segment_atom_attack(ot_bf_seg_dict,
                                              my_atom_bf_seg_dict,
                                              my_bit_pos_q_gram_dict,
                                              all_q_gram_attr_val_dict,
                                              num_bf_to_attack)

  res_str += '%d, %d, %d, %d, %d, ' % (num_corr_1_1_attr_matches, \
                                     num_corr_1_m_attr_matches, \
                                     num_wrong_1_matches, num_wrong_m_matches, \
                                     num_no_matches)

  # profile 
  #
  num_reidentified_ids,num_reidentified_ids_matches, \
  num_reidentified_ids_no_matches = profile_reidentied_records(DATASET_PATH,
                                          my_data_set_name,ot_data_set_name,
                                          col_sep_char,rec_id_col, attr_list)
  
  res_str += '%d, %d, %d,' % (num_reidentified_ids,num_reidentified_ids_matches,
                              num_reidentified_ids_no_matches)
  
  saida_lista = ''
  for r_id in records_matched:
    saida_lista += str(r_id)+'#'

  res_str += '%s' % (saida_lista) 
  # Print result line for CSV generation
  #

  print(res_str)
  f.write(res_str.split('### ')[1]+'\n') #savinf csv file
  f.flush()

f.close()


In [ ]:
#@title Run attack { form-width: "50%" }

# NUM_MATCHES_TO_TRY = 1000  # or 'all'

# initial parameters
BF_HASH_FUNCT1 = hashlib.sha1
BF_HASH_FUNCT2 = hashlib.md5
random.seed(42)

DATASET_PATH='/tmp/'
try:
  RESULTS_PATH
except NameError:
  RESULTS_PATH='/tmp/'

command_line_call = sys.argv


#@markdown -----------------
#@markdown ##### Dataset:
dataset = "ncvoter-sample-nm-s3" #@param ["ncvoter","ncvoter-sample","ncvoter-sample-nm-s2","ncvoter-sample-nm-s3","ncvoter-sample-nm-s4","ncvoter-sample-nm-s5","ncvoter-sample-nm-s6","politicians","politicians-split-2","politicians-split-3","politicians-split-4","politicians-split-5","politicians-split-6"]

if dataset == 'politicians':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-all-1to1-b.csv.gz'
if dataset == 'politicians-split-2':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i2.csv.gz'
if dataset == 'politicians-split-3':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i3.csv.gz'
if dataset == 'politicians-split-4':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i4.csv.gz'
if dataset == 'politicians-split-5':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i5.csv.gz'
if dataset == 'politicians-split-6':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i6.csv.gz'

if dataset == 'ncvoter':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-b.csv.gz'
if dataset == 'ncvoter-sample':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b.csv.gz'
if dataset == 'ncvoter-sample-nm-s2':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split2.csv.gz'
if dataset == 'ncvoter-sample-nm-s3':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split3.csv.gz'
if dataset == 'ncvoter-sample-nm-s4':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split4.csv.gz'
if dataset == 'ncvoter-sample-nm-s5':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split5.csv.gz'
if dataset == 'ncvoter-sample-nm-s6':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split6.csv.gz'

rec_id_col   = 0   #@param {type:"integer"}

#voter_reg_num,name_prefix,first_name,middle_name,last_name,
#name_suffix,age,gender,race,ethnic,
#street_address,city,state,zip_code,full_phone_num,
#birth_place,register_date,download_month
#@markdown ###### attr_list parameters
#@markdown | ATTRS | POS | ATTRS | POS |
#@markdown |:---:|:---:|:---:|:---:|
#@markdown |<td colspan=2>NCVR <td colspan=2>BR
#@markdown |first_name |3|first_name |1|
#@markdown |first_name , last_name|3,5|first_name , last_name|1,2|
#@markdown | full_name |3,4,5| full_name |3|
#@markdown |first_name , last_name , address|3,5,11|full_name, address|3,9,10|
# <sup> NCVR </sup>
# > <sup>3,5 - first_name , last_name</sup>\
# > <sup>3,5,11 - first_name , last_name, street_address</sup>\
# > <sup>3,5,6,7,12 - first_name , last_name, age, gender, city</sup>
attr_list        =    [3,5]#@param {type:"raw"}
#@markdown ###### col separator [(',' - ncvr),(';' - brpol)]
col_sep_char = ',' #@param {type:"string"}[',',';']
header_line_flag = True #@param {type:"raw"}

#@markdown -----------------
#@markdown #### Anonymization Parameter:

q =  2#@param {type:"integer"}
hash_type = "xrh" #@param ["rh","xrh","xbf","dh"]
num_hash_funct = "opt" #@param ["opt", "opt_half", "opt_quarter"]
bf_len =  200#@param {type:"integer"}

#ajustar depois
bf_seg_perc_list = [15] #@param {type:"raw"}
num_bf_to_attack =  'all'#@param {type:"raw"}

main_start_time = time.time()

# command_line_call = sys.argv
# q =                int(sys.argv[1])
# hash_type =        sys.argv[2].lower()
# num_hash_funct =   sys.argv[3]
# bf_len =           int(sys.argv[4])
# my_data_set_name = sys.argv[5]
# ot_data_set_name = sys.argv[6]
# rec_id_col =       int(sys.argv[7])
# col_sep_char =     sys.argv[8]
# header_line_flag = eval(sys.argv[9])
# attr_list =        eval(sys.argv[10])
# bf_seg_perc_list = eval(sys.argv[11])
# num_bf_to_attack = sys.argv[12]

assert q >= 1, q
assert hash_type in ['dh','rh','xrh','xbf'], hash_type
if num_hash_funct.isdigit():
  num_hash_funct = int(num_hash_funct)
  assert num_hash_funct >= 1, num_hash_funct
else:
  assert num_hash_funct in ['opt', 'opt_half', 'opt_quarter'], num_hash_funct
assert bf_len > 1, bf_len
#
assert rec_id_col >= 0, rec_id_col
assert header_line_flag in [True,False], header_line_flag
assert isinstance(attr_list, list), attr_list
assert isinstance(bf_seg_perc_list, list), bf_seg_perc_list
if (num_bf_to_attack != 'all'):
  num_bf_to_attack = int(num_bf_to_attack)
  assert num_bf_to_attack > 1, num_bf_to_attack

my_base_data_set_name = my_data_set_name.split('/')[-1].replace('.csv', '')
my_base_data_set_name = my_base_data_set_name.replace('.gz','')
ot_base_data_set_name = ot_data_set_name.split('/')[-1].replace('.csv', '')
ot_base_data_set_name = ot_base_data_set_name.replace('.gz','')

assert ',' not in my_base_data_set_name
assert ',' not in ot_base_data_set_name

print('')
print('-'*80)
print('')

ds_name = my_data_set_name.split('-')[0]
res_file = 'real-'+ds_name+'-'+str(num_bf_to_attack)+'-'+str(hash_type)+'-'+str(bf_len)+'.csv'
result_file_exists = os.path.exists(RESULTS_PATH + res_file)

f = open(RESULTS_PATH + res_file, "a")



# -----------------------------------------------------------------------------
# Step 1: Load the data sets and extract q-grams for selected attributes
#
my_attr_val_q_gram_dict, my_q_gram_attr_val_dict, my_avr_num_q_gram, \
   my_all_q_gram_set, my_encoded_attr_name_str = \
                       load_data_set_extract_q_grams(my_data_set_name,
                                                     rec_id_col, attr_list,
                                                     col_sep_char,
                                                     header_line_flag, q)

ot_attr_val_q_gram_dict, ot_q_gram_attr_val_dict, ot_avr_num_q_gram, \
   ot_all_q_gram_set, ot_encoded_attr_name_str = \
                       load_data_set_extract_q_grams(ot_data_set_name,
                                                     rec_id_col, attr_list,
                                                     col_sep_char,
                                                     header_line_flag, q)

assert my_encoded_attr_name_str == ot_encoded_attr_name_str

# Generate a dictionary with all q-grams and their attribute values from both
# data sets
#
my_num_q_gram = len(my_q_gram_attr_val_dict)
ot_num_q_gram = len(ot_q_gram_attr_val_dict)

all_q_gram_attr_val_dict =    {}

#modified to python3
for (q_gram, attr_val_set) in my_q_gram_attr_val_dict.items():
  all_q_gram_attr_val_dict[q_gram] = attr_val_set
for (q_gram, attr_val_set) in ot_q_gram_attr_val_dict.items():
  q_gram_attr_val_set = all_q_gram_attr_val_dict.get(q_gram, set())
  q_gram_attr_val_set = q_gram_attr_val_set | attr_val_set
  all_q_gram_attr_val_dict[q_gram] = q_gram_attr_val_set

num_all_q_gram =    len(all_q_gram_attr_val_dict)
num_common_q_gram = len(set(my_q_gram_attr_val_dict.keys()) & \
                        set(ot_q_gram_attr_val_dict.keys()))

print('A total of %d unique q-grams occur in both data set' % (num_all_q_gram))
print('  Of these, %d occur in both data sets' % (num_common_q_gram))
print('')

num_common_attr_val = len(set(my_attr_val_q_gram_dict.keys()) & \
                          set(ot_attr_val_q_gram_dict.keys()))
num_my_attr_val = len(my_attr_val_q_gram_dict)
num_ot_attr_val = len(ot_attr_val_q_gram_dict)

jacc_common_attr_val = float(num_common_attr_val) / \
  (num_my_attr_val + num_ot_attr_val - num_common_attr_val)

print('My database contains %d unique attribute values' % (num_my_attr_val))
print('The other database contains %d unique attribute values' % \
      (num_ot_attr_val))
print('  Number of unique attribute values in common: %d' % \
      (num_common_attr_val))
print('    Jaccard based overlap of common attribute values: %.1f%%' % \
      (100.0*jacc_common_attr_val))
print('')

# -----------------------------------------------------------------------------
# Step 2: Generate Bloom filters from q-gram sets
#
if (num_hash_funct in ['opt', 'opt_half', 'opt_quarter']):

  # Set number of hash functions to have in average 50% of bits set to 1
  # (Linking Sensitive Data book, 2020)
  # num_hash_funct = int(math.ceil(0.5 * BF_LEN / \
  #                                math.floor(avrg_num_q_gram)))
  #
  opt_num_hash_funct = int(round(numpy.log(2.0) * float(bf_len) / \
                                 my_avr_num_q_gram))
  if (num_hash_funct == 'opt'):
    num_hash_funct = opt_num_hash_funct
  elif (num_hash_funct == 'opt_half'):
    num_hash_funct = int(round(opt_num_hash_funct / 2.0))
  elif (num_hash_funct == 'opt_quarter'):
    num_hash_funct = int(round(opt_num_hash_funct / 4.0))
  else:
    raise Exception(num_hash_funct)

my_bf_dict, my_bit_pos_q_gram_dict, my_atom_bf_dict = \
    gen_bloom_filter_dict(my_attr_val_q_gram_dict, hash_type, bf_len,
                          num_hash_funct)


ot_bf_dict, ot_bit_pos_q_gram_dict, ot_atom_bf_dict = \
    gen_bloom_filter_dict(ot_attr_val_q_gram_dict, hash_type, bf_len,
                          num_hash_funct)

#rodando ate aqui

# Header string for results output
#
header = '### command_line_call, encoded_attr_name_str,num_bf_to_attack, ' +\
      'hash_type, num_hash_funct, num_my_attr_val, num_ot_attr_val, ' +\
      'num_common_attr_val, jacc_common_perc, my_num_q_gram, ot_num_q_gram, ' +\
      'num_all_q_gram, num_common_q_gram, bf_seg_len, bf_seg_perc, ' +\
      'my_num_q_gram_bf_seg, my_min_num_bit_pos, my_avr_num_bit_pos, ' +\
      'my_med_num_bit_pos, my_max_num_bit_pos, ot_num_q_gram_bf_seg, ' +\
      'ot_min_num_bit_pos, ot_avr_num_bit_pos, ot_med_num_bit_pos, ' +\
      'ot_max_num_bit_pos, atom_num_corr_1_1_attr_matches, ' +\
      'atom_num_corr_1_m_attr_matches, atom_num_wrong_1_matches, ' +\
      'atom_num_wrong_m_matches, atom_num_no_matches, num_reidentified_ids, ' +\
      'num_reidentified_ids_matches, num_reidentified_ids_no_matches, ' +\
      'reidentified_id_list'

print(header)

if result_file_exists == False:
  f.write(header.split('### ')[1]+'\n')

# -----------------------------------------------------------------------------
# Step 3: Run the segment attack with different segment percentages
#
for bf_seg_perc in bf_seg_perc_list:

  print('-'*80)
  print()

  # Generate a string to be printed as the result summary
  #
  res_str = '### "' + ' '.join(command_line_call)
  res_str = res_str.replace(' , ', ' comma ')  # The separator in input file
  res_str = res_str.replace(',', ';') + '", '

  # Details of how many attribute values in the two data sets, and how many
  # occur in common in both data sets
  #
  res_str += '%s, %s, %s, %d, %d, %d, %d, %.1f, ' % (my_encoded_attr_name_str,
                                                str(num_bf_to_attack),
                                                hash_type,
                                                num_hash_funct, num_my_attr_val,
                                                num_ot_attr_val,
                                                num_common_attr_val,
                                                100.0*jacc_common_attr_val)

  # Details of how many q-grams in the two data sets, how many in total,
  # and how many occur occur in common in both data sets
  #
  res_str += '%d, %d, %d, %d, ' % (my_num_q_gram, ot_num_q_gram, \
                                   num_all_q_gram, num_common_q_gram)

  if hash_type == 'xbf':
    bf_seg_len = int((float(bf_len)/2)*bf_seg_perc/100)
  else:
    bf_seg_len = int(float(bf_len)*bf_seg_perc/100)

  # Get the BF segments from the BFs of both data sets
  #
  my_bf_seg_dict = get_bf_segments(my_bf_dict, bf_seg_len)
  ot_bf_seg_dict = get_bf_segments(ot_bf_dict, bf_seg_len)
  
  my_atom_bf_seg_dict = get_bf_segments(my_atom_bf_dict, bf_seg_len)
  
  my_num_q_gram_bf_seg, my_min_num_bit_pos, my_avr_num_bit_pos, \
         my_med_num_bit_pos, my_max_num_bit_pos = \
                  bf_segment_get_num_q_gram(my_bit_pos_q_gram_dict,
                                            bf_seg_len, num_hash_funct)
  ot_num_q_gram_bf_seg, ot_min_num_bit_pos, ot_avr_num_bit_pos, \
         ot_med_num_bit_pos, ot_max_num_bit_pos = \
                  bf_segment_get_num_q_gram(ot_bit_pos_q_gram_dict,
                                            bf_seg_len, num_hash_funct)

  # Add BF segment information to result string
  #
  res_str += '%d, %d%%, %d, %d, %.2f, %d, %d, ' % (bf_seg_len, bf_seg_perc, \
                                                 my_num_q_gram_bf_seg, \
                                                 my_min_num_bit_pos, \
                                                 my_avr_num_bit_pos, \
                                                 my_med_num_bit_pos, \
                                                 my_max_num_bit_pos)
  res_str += '%d, %d, %.2f, %d, %d, ' % (ot_num_q_gram_bf_seg, \
                                       ot_min_num_bit_pos, \
                                       ot_avr_num_bit_pos, \
                                       ot_med_num_bit_pos, \
                                       ot_max_num_bit_pos)

  # Atom based attack
  #
  num_corr_1_1_attr_matches, num_corr_1_m_attr_matches, \
      num_wrong_1_matches, num_wrong_m_matches, \
      num_no_matches,records_matched = bf_segment_atom_attack(ot_bf_seg_dict,
                                              my_atom_bf_seg_dict,
                                              my_bit_pos_q_gram_dict,
                                              all_q_gram_attr_val_dict,
                                              num_bf_to_attack)

  res_str += '%d, %d, %d, %d, %d, ' % (num_corr_1_1_attr_matches, \
                                     num_corr_1_m_attr_matches, \
                                     num_wrong_1_matches, num_wrong_m_matches, \
                                     num_no_matches)

  # profile 
  #
  num_reidentified_ids,num_reidentified_ids_matches, \
  num_reidentified_ids_no_matches = profile_reidentied_records(DATASET_PATH,
                                          my_data_set_name,ot_data_set_name,
                                          col_sep_char,rec_id_col, attr_list)
  
  res_str += '%d, %d, %d,' % (num_reidentified_ids,num_reidentified_ids_matches,
                              num_reidentified_ids_no_matches)
  
  saida_lista = ''
  for r_id in records_matched:
    saida_lista += str(r_id)+'#'

  res_str += '%s' % (saida_lista) 
  # Print result line for CSV generation
  #

  print(res_str)
  f.write(res_str.split('### ')[1]+'\n') #savinf csv file
  f.flush()

f.close()


In [ ]:
#@title Run attack { form-width: "50%" }

# NUM_MATCHES_TO_TRY = 1000  # or 'all'

# initial parameters
BF_HASH_FUNCT1 = hashlib.sha1
BF_HASH_FUNCT2 = hashlib.md5
random.seed(42)

DATASET_PATH='/tmp/'
try:
  RESULTS_PATH
except NameError:
  RESULTS_PATH='/tmp/'

command_line_call = sys.argv


#@markdown -----------------
#@markdown ##### Dataset:
dataset = "ncvoter-sample-nm-s4" #@param ["ncvoter","ncvoter-sample","ncvoter-sample-nm-s2","ncvoter-sample-nm-s3","ncvoter-sample-nm-s4","ncvoter-sample-nm-s5","ncvoter-sample-nm-s6","politicians","politicians-split-2","politicians-split-3","politicians-split-4","politicians-split-5","politicians-split-6"]

if dataset == 'politicians':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-all-1to1-b.csv.gz'
if dataset == 'politicians-split-2':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i2.csv.gz'
if dataset == 'politicians-split-3':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i3.csv.gz'
if dataset == 'politicians-split-4':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i4.csv.gz'
if dataset == 'politicians-split-5':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i5.csv.gz'
if dataset == 'politicians-split-6':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i6.csv.gz'

if dataset == 'ncvoter':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-b.csv.gz'
if dataset == 'ncvoter-sample':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b.csv.gz'
if dataset == 'ncvoter-sample-nm-s2':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split2.csv.gz'
if dataset == 'ncvoter-sample-nm-s3':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split3.csv.gz'
if dataset == 'ncvoter-sample-nm-s4':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split4.csv.gz'
if dataset == 'ncvoter-sample-nm-s5':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split5.csv.gz'
if dataset == 'ncvoter-sample-nm-s6':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split6.csv.gz'

rec_id_col   = 0   #@param {type:"integer"}

#voter_reg_num,name_prefix,first_name,middle_name,last_name,
#name_suffix,age,gender,race,ethnic,
#street_address,city,state,zip_code,full_phone_num,
#birth_place,register_date,download_month
#@markdown ###### attr_list parameters
#@markdown | ATTRS | POS | ATTRS | POS |
#@markdown |:---:|:---:|:---:|:---:|
#@markdown |<td colspan=2>NCVR <td colspan=2>BR
#@markdown |first_name |3|first_name |1|
#@markdown |first_name , last_name|3,5|first_name , last_name|1,2|
#@markdown | full_name |3,4,5| full_name |3|
#@markdown |first_name , last_name , address|3,5,11|full_name, address|3,9,10|
# <sup> NCVR </sup>
# > <sup>3,5 - first_name , last_name</sup>\
# > <sup>3,5,11 - first_name , last_name, street_address</sup>\
# > <sup>3,5,6,7,12 - first_name , last_name, age, gender, city</sup>
attr_list        =    [3,5]#@param {type:"raw"}
#@markdown ###### col separator [(',' - ncvr),(';' - brpol)]
col_sep_char = ',' #@param {type:"string"}[',',';']
header_line_flag = True #@param {type:"raw"}

#@markdown -----------------
#@markdown #### Anonymization Parameter:

q =  2#@param {type:"integer"}
hash_type = "xrh" #@param ["rh","xrh","xbf","dh"]
num_hash_funct = "opt" #@param ["opt", "opt_half", "opt_quarter"]
bf_len =  200#@param {type:"integer"}

#ajustar depois
bf_seg_perc_list = [20] #@param {type:"raw"}
num_bf_to_attack =  'all'#@param {type:"raw"}

main_start_time = time.time()

# command_line_call = sys.argv
# q =                int(sys.argv[1])
# hash_type =        sys.argv[2].lower()
# num_hash_funct =   sys.argv[3]
# bf_len =           int(sys.argv[4])
# my_data_set_name = sys.argv[5]
# ot_data_set_name = sys.argv[6]
# rec_id_col =       int(sys.argv[7])
# col_sep_char =     sys.argv[8]
# header_line_flag = eval(sys.argv[9])
# attr_list =        eval(sys.argv[10])
# bf_seg_perc_list = eval(sys.argv[11])
# num_bf_to_attack = sys.argv[12]

assert q >= 1, q
assert hash_type in ['dh','rh','xrh','xbf'], hash_type
if num_hash_funct.isdigit():
  num_hash_funct = int(num_hash_funct)
  assert num_hash_funct >= 1, num_hash_funct
else:
  assert num_hash_funct in ['opt', 'opt_half', 'opt_quarter'], num_hash_funct
assert bf_len > 1, bf_len
#
assert rec_id_col >= 0, rec_id_col
assert header_line_flag in [True,False], header_line_flag
assert isinstance(attr_list, list), attr_list
assert isinstance(bf_seg_perc_list, list), bf_seg_perc_list
if (num_bf_to_attack != 'all'):
  num_bf_to_attack = int(num_bf_to_attack)
  assert num_bf_to_attack > 1, num_bf_to_attack

my_base_data_set_name = my_data_set_name.split('/')[-1].replace('.csv', '')
my_base_data_set_name = my_base_data_set_name.replace('.gz','')
ot_base_data_set_name = ot_data_set_name.split('/')[-1].replace('.csv', '')
ot_base_data_set_name = ot_base_data_set_name.replace('.gz','')

assert ',' not in my_base_data_set_name
assert ',' not in ot_base_data_set_name

print('')
print('-'*80)
print('')

ds_name = my_data_set_name.split('-')[0]
res_file = 'real-'+ds_name+'-'+str(num_bf_to_attack)+'-'+str(hash_type)+'-'+str(bf_len)+'.csv'
result_file_exists = os.path.exists(RESULTS_PATH + res_file)

f = open(RESULTS_PATH + res_file, "a")



# -----------------------------------------------------------------------------
# Step 1: Load the data sets and extract q-grams for selected attributes
#
my_attr_val_q_gram_dict, my_q_gram_attr_val_dict, my_avr_num_q_gram, \
   my_all_q_gram_set, my_encoded_attr_name_str = \
                       load_data_set_extract_q_grams(my_data_set_name,
                                                     rec_id_col, attr_list,
                                                     col_sep_char,
                                                     header_line_flag, q)

ot_attr_val_q_gram_dict, ot_q_gram_attr_val_dict, ot_avr_num_q_gram, \
   ot_all_q_gram_set, ot_encoded_attr_name_str = \
                       load_data_set_extract_q_grams(ot_data_set_name,
                                                     rec_id_col, attr_list,
                                                     col_sep_char,
                                                     header_line_flag, q)

assert my_encoded_attr_name_str == ot_encoded_attr_name_str

# Generate a dictionary with all q-grams and their attribute values from both
# data sets
#
my_num_q_gram = len(my_q_gram_attr_val_dict)
ot_num_q_gram = len(ot_q_gram_attr_val_dict)

all_q_gram_attr_val_dict =    {}

#modified to python3
for (q_gram, attr_val_set) in my_q_gram_attr_val_dict.items():
  all_q_gram_attr_val_dict[q_gram] = attr_val_set
for (q_gram, attr_val_set) in ot_q_gram_attr_val_dict.items():
  q_gram_attr_val_set = all_q_gram_attr_val_dict.get(q_gram, set())
  q_gram_attr_val_set = q_gram_attr_val_set | attr_val_set
  all_q_gram_attr_val_dict[q_gram] = q_gram_attr_val_set

num_all_q_gram =    len(all_q_gram_attr_val_dict)
num_common_q_gram = len(set(my_q_gram_attr_val_dict.keys()) & \
                        set(ot_q_gram_attr_val_dict.keys()))

print('A total of %d unique q-grams occur in both data set' % (num_all_q_gram))
print('  Of these, %d occur in both data sets' % (num_common_q_gram))
print('')

num_common_attr_val = len(set(my_attr_val_q_gram_dict.keys()) & \
                          set(ot_attr_val_q_gram_dict.keys()))
num_my_attr_val = len(my_attr_val_q_gram_dict)
num_ot_attr_val = len(ot_attr_val_q_gram_dict)

jacc_common_attr_val = float(num_common_attr_val) / \
  (num_my_attr_val + num_ot_attr_val - num_common_attr_val)

print('My database contains %d unique attribute values' % (num_my_attr_val))
print('The other database contains %d unique attribute values' % \
      (num_ot_attr_val))
print('  Number of unique attribute values in common: %d' % \
      (num_common_attr_val))
print('    Jaccard based overlap of common attribute values: %.1f%%' % \
      (100.0*jacc_common_attr_val))
print('')

# -----------------------------------------------------------------------------
# Step 2: Generate Bloom filters from q-gram sets
#
if (num_hash_funct in ['opt', 'opt_half', 'opt_quarter']):

  # Set number of hash functions to have in average 50% of bits set to 1
  # (Linking Sensitive Data book, 2020)
  # num_hash_funct = int(math.ceil(0.5 * BF_LEN / \
  #                                math.floor(avrg_num_q_gram)))
  #
  opt_num_hash_funct = int(round(numpy.log(2.0) * float(bf_len) / \
                                 my_avr_num_q_gram))
  if (num_hash_funct == 'opt'):
    num_hash_funct = opt_num_hash_funct
  elif (num_hash_funct == 'opt_half'):
    num_hash_funct = int(round(opt_num_hash_funct / 2.0))
  elif (num_hash_funct == 'opt_quarter'):
    num_hash_funct = int(round(opt_num_hash_funct / 4.0))
  else:
    raise Exception(num_hash_funct)

my_bf_dict, my_bit_pos_q_gram_dict, my_atom_bf_dict = \
    gen_bloom_filter_dict(my_attr_val_q_gram_dict, hash_type, bf_len,
                          num_hash_funct)


ot_bf_dict, ot_bit_pos_q_gram_dict, ot_atom_bf_dict = \
    gen_bloom_filter_dict(ot_attr_val_q_gram_dict, hash_type, bf_len,
                          num_hash_funct)

#rodando ate aqui

# Header string for results output
#
header = '### command_line_call, encoded_attr_name_str,num_bf_to_attack, ' +\
      'hash_type, num_hash_funct, num_my_attr_val, num_ot_attr_val, ' +\
      'num_common_attr_val, jacc_common_perc, my_num_q_gram, ot_num_q_gram, ' +\
      'num_all_q_gram, num_common_q_gram, bf_seg_len, bf_seg_perc, ' +\
      'my_num_q_gram_bf_seg, my_min_num_bit_pos, my_avr_num_bit_pos, ' +\
      'my_med_num_bit_pos, my_max_num_bit_pos, ot_num_q_gram_bf_seg, ' +\
      'ot_min_num_bit_pos, ot_avr_num_bit_pos, ot_med_num_bit_pos, ' +\
      'ot_max_num_bit_pos, atom_num_corr_1_1_attr_matches, ' +\
      'atom_num_corr_1_m_attr_matches, atom_num_wrong_1_matches, ' +\
      'atom_num_wrong_m_matches, atom_num_no_matches, num_reidentified_ids, ' +\
      'num_reidentified_ids_matches, num_reidentified_ids_no_matches, ' +\
      'reidentified_id_list'

print(header)

if result_file_exists == False:
  f.write(header.split('### ')[1]+'\n')

# -----------------------------------------------------------------------------
# Step 3: Run the segment attack with different segment percentages
#
for bf_seg_perc in bf_seg_perc_list:

  print('-'*80)
  print()

  # Generate a string to be printed as the result summary
  #
  res_str = '### "' + ' '.join(command_line_call)
  res_str = res_str.replace(' , ', ' comma ')  # The separator in input file
  res_str = res_str.replace(',', ';') + '", '

  # Details of how many attribute values in the two data sets, and how many
  # occur in common in both data sets
  #
  res_str += '%s, %s, %s, %d, %d, %d, %d, %.1f, ' % (my_encoded_attr_name_str,
                                                str(num_bf_to_attack),
                                                hash_type,
                                                num_hash_funct, num_my_attr_val,
                                                num_ot_attr_val,
                                                num_common_attr_val,
                                                100.0*jacc_common_attr_val)

  # Details of how many q-grams in the two data sets, how many in total,
  # and how many occur occur in common in both data sets
  #
  res_str += '%d, %d, %d, %d, ' % (my_num_q_gram, ot_num_q_gram, \
                                   num_all_q_gram, num_common_q_gram)

  if hash_type == 'xbf':
    bf_seg_len = int((float(bf_len)/2)*bf_seg_perc/100)
  else:
    bf_seg_len = int(float(bf_len)*bf_seg_perc/100)

  # Get the BF segments from the BFs of both data sets
  #
  my_bf_seg_dict = get_bf_segments(my_bf_dict, bf_seg_len)
  ot_bf_seg_dict = get_bf_segments(ot_bf_dict, bf_seg_len)
  
  my_atom_bf_seg_dict = get_bf_segments(my_atom_bf_dict, bf_seg_len)
  
  my_num_q_gram_bf_seg, my_min_num_bit_pos, my_avr_num_bit_pos, \
         my_med_num_bit_pos, my_max_num_bit_pos = \
                  bf_segment_get_num_q_gram(my_bit_pos_q_gram_dict,
                                            bf_seg_len, num_hash_funct)
  ot_num_q_gram_bf_seg, ot_min_num_bit_pos, ot_avr_num_bit_pos, \
         ot_med_num_bit_pos, ot_max_num_bit_pos = \
                  bf_segment_get_num_q_gram(ot_bit_pos_q_gram_dict,
                                            bf_seg_len, num_hash_funct)

  # Add BF segment information to result string
  #
  res_str += '%d, %d%%, %d, %d, %.2f, %d, %d, ' % (bf_seg_len, bf_seg_perc, \
                                                 my_num_q_gram_bf_seg, \
                                                 my_min_num_bit_pos, \
                                                 my_avr_num_bit_pos, \
                                                 my_med_num_bit_pos, \
                                                 my_max_num_bit_pos)
  res_str += '%d, %d, %.2f, %d, %d, ' % (ot_num_q_gram_bf_seg, \
                                       ot_min_num_bit_pos, \
                                       ot_avr_num_bit_pos, \
                                       ot_med_num_bit_pos, \
                                       ot_max_num_bit_pos)

  # Atom based attack
  #
  num_corr_1_1_attr_matches, num_corr_1_m_attr_matches, \
      num_wrong_1_matches, num_wrong_m_matches, \
      num_no_matches,records_matched = bf_segment_atom_attack(ot_bf_seg_dict,
                                              my_atom_bf_seg_dict,
                                              my_bit_pos_q_gram_dict,
                                              all_q_gram_attr_val_dict,
                                              num_bf_to_attack)

  res_str += '%d, %d, %d, %d, %d, ' % (num_corr_1_1_attr_matches, \
                                     num_corr_1_m_attr_matches, \
                                     num_wrong_1_matches, num_wrong_m_matches, \
                                     num_no_matches)

  # profile 
  #
  num_reidentified_ids,num_reidentified_ids_matches, \
  num_reidentified_ids_no_matches = profile_reidentied_records(DATASET_PATH,
                                          my_data_set_name,ot_data_set_name,
                                          col_sep_char,rec_id_col, attr_list)
  
  res_str += '%d, %d, %d,' % (num_reidentified_ids,num_reidentified_ids_matches,
                              num_reidentified_ids_no_matches)
  
  saida_lista = ''
  for r_id in records_matched:
    saida_lista += str(r_id)+'#'

  res_str += '%s' % (saida_lista) 
  # Print result line for CSV generation
  #

  print(res_str)
  f.write(res_str.split('### ')[1]+'\n') #savinf csv file
  f.flush()

f.close()


In [ ]:
#@title Run attack { form-width: "50%" }

# NUM_MATCHES_TO_TRY = 1000  # or 'all'

# initial parameters
BF_HASH_FUNCT1 = hashlib.sha1
BF_HASH_FUNCT2 = hashlib.md5
random.seed(42)

DATASET_PATH='/tmp/'
try:
  RESULTS_PATH
except NameError:
  RESULTS_PATH='/tmp/'

command_line_call = sys.argv


#@markdown -----------------
#@markdown ##### Dataset:
dataset = "ncvoter-sample-nm-s5" #@param ["ncvoter","ncvoter-sample","ncvoter-sample-nm-s2","ncvoter-sample-nm-s3","ncvoter-sample-nm-s4","ncvoter-sample-nm-s5","ncvoter-sample-nm-s6","politicians","politicians-split-2","politicians-split-3","politicians-split-4","politicians-split-5","politicians-split-6"]

if dataset == 'politicians':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-all-1to1-b.csv.gz'
if dataset == 'politicians-split-2':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i2.csv.gz'
if dataset == 'politicians-split-3':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i3.csv.gz'
if dataset == 'politicians-split-4':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i4.csv.gz'
if dataset == 'politicians-split-5':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i5.csv.gz'
if dataset == 'politicians-split-6':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i6.csv.gz'

if dataset == 'ncvoter':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-b.csv.gz'
if dataset == 'ncvoter-sample':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b.csv.gz'
if dataset == 'ncvoter-sample-nm-s2':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split2.csv.gz'
if dataset == 'ncvoter-sample-nm-s3':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split3.csv.gz'
if dataset == 'ncvoter-sample-nm-s4':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split4.csv.gz'
if dataset == 'ncvoter-sample-nm-s5':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split5.csv.gz'
if dataset == 'ncvoter-sample-nm-s6':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split6.csv.gz'

rec_id_col   = 0   #@param {type:"integer"}

#voter_reg_num,name_prefix,first_name,middle_name,last_name,
#name_suffix,age,gender,race,ethnic,
#street_address,city,state,zip_code,full_phone_num,
#birth_place,register_date,download_month
#@markdown ###### attr_list parameters
#@markdown | ATTRS | POS | ATTRS | POS |
#@markdown |:---:|:---:|:---:|:---:|
#@markdown |<td colspan=2>NCVR <td colspan=2>BR
#@markdown |first_name |3|first_name |1|
#@markdown |first_name , last_name|3,5|first_name , last_name|1,2|
#@markdown | full_name |3,4,5| full_name |3|
#@markdown |first_name , last_name , address|3,5,11|full_name, address|3,9,10|
# <sup> NCVR </sup>
# > <sup>3,5 - first_name , last_name</sup>\
# > <sup>3,5,11 - first_name , last_name, street_address</sup>\
# > <sup>3,5,6,7,12 - first_name , last_name, age, gender, city</sup>
attr_list        =    [3,5]#@param {type:"raw"}
#@markdown ###### col separator [(',' - ncvr),(';' - brpol)]
col_sep_char = ',' #@param {type:"string"}[',',';']
header_line_flag = True #@param {type:"raw"}

#@markdown -----------------
#@markdown #### Anonymization Parameter:

q =  2#@param {type:"integer"}
hash_type = "xrh" #@param ["rh","xrh","xbf","dh"]
num_hash_funct = "opt" #@param ["opt", "opt_half", "opt_quarter"]
bf_len =  200#@param {type:"integer"}

#ajustar depois
bf_seg_perc_list = [25] #@param {type:"raw"}
num_bf_to_attack =  'all'#@param {type:"raw"}

main_start_time = time.time()

# command_line_call = sys.argv
# q =                int(sys.argv[1])
# hash_type =        sys.argv[2].lower()
# num_hash_funct =   sys.argv[3]
# bf_len =           int(sys.argv[4])
# my_data_set_name = sys.argv[5]
# ot_data_set_name = sys.argv[6]
# rec_id_col =       int(sys.argv[7])
# col_sep_char =     sys.argv[8]
# header_line_flag = eval(sys.argv[9])
# attr_list =        eval(sys.argv[10])
# bf_seg_perc_list = eval(sys.argv[11])
# num_bf_to_attack = sys.argv[12]

assert q >= 1, q
assert hash_type in ['dh','rh','xrh','xbf'], hash_type
if num_hash_funct.isdigit():
  num_hash_funct = int(num_hash_funct)
  assert num_hash_funct >= 1, num_hash_funct
else:
  assert num_hash_funct in ['opt', 'opt_half', 'opt_quarter'], num_hash_funct
assert bf_len > 1, bf_len
#
assert rec_id_col >= 0, rec_id_col
assert header_line_flag in [True,False], header_line_flag
assert isinstance(attr_list, list), attr_list
assert isinstance(bf_seg_perc_list, list), bf_seg_perc_list
if (num_bf_to_attack != 'all'):
  num_bf_to_attack = int(num_bf_to_attack)
  assert num_bf_to_attack > 1, num_bf_to_attack

my_base_data_set_name = my_data_set_name.split('/')[-1].replace('.csv', '')
my_base_data_set_name = my_base_data_set_name.replace('.gz','')
ot_base_data_set_name = ot_data_set_name.split('/')[-1].replace('.csv', '')
ot_base_data_set_name = ot_base_data_set_name.replace('.gz','')

assert ',' not in my_base_data_set_name
assert ',' not in ot_base_data_set_name

print('')
print('-'*80)
print('')

ds_name = my_data_set_name.split('-')[0]
res_file = 'real-'+ds_name+'-'+str(num_bf_to_attack)+'-'+str(hash_type)+'-'+str(bf_len)+'.csv'
result_file_exists = os.path.exists(RESULTS_PATH + res_file)

f = open(RESULTS_PATH + res_file, "a")



# -----------------------------------------------------------------------------
# Step 1: Load the data sets and extract q-grams for selected attributes
#
my_attr_val_q_gram_dict, my_q_gram_attr_val_dict, my_avr_num_q_gram, \
   my_all_q_gram_set, my_encoded_attr_name_str = \
                       load_data_set_extract_q_grams(my_data_set_name,
                                                     rec_id_col, attr_list,
                                                     col_sep_char,
                                                     header_line_flag, q)

ot_attr_val_q_gram_dict, ot_q_gram_attr_val_dict, ot_avr_num_q_gram, \
   ot_all_q_gram_set, ot_encoded_attr_name_str = \
                       load_data_set_extract_q_grams(ot_data_set_name,
                                                     rec_id_col, attr_list,
                                                     col_sep_char,
                                                     header_line_flag, q)

assert my_encoded_attr_name_str == ot_encoded_attr_name_str

# Generate a dictionary with all q-grams and their attribute values from both
# data sets
#
my_num_q_gram = len(my_q_gram_attr_val_dict)
ot_num_q_gram = len(ot_q_gram_attr_val_dict)

all_q_gram_attr_val_dict =    {}

#modified to python3
for (q_gram, attr_val_set) in my_q_gram_attr_val_dict.items():
  all_q_gram_attr_val_dict[q_gram] = attr_val_set
for (q_gram, attr_val_set) in ot_q_gram_attr_val_dict.items():
  q_gram_attr_val_set = all_q_gram_attr_val_dict.get(q_gram, set())
  q_gram_attr_val_set = q_gram_attr_val_set | attr_val_set
  all_q_gram_attr_val_dict[q_gram] = q_gram_attr_val_set

num_all_q_gram =    len(all_q_gram_attr_val_dict)
num_common_q_gram = len(set(my_q_gram_attr_val_dict.keys()) & \
                        set(ot_q_gram_attr_val_dict.keys()))

print('A total of %d unique q-grams occur in both data set' % (num_all_q_gram))
print('  Of these, %d occur in both data sets' % (num_common_q_gram))
print('')

num_common_attr_val = len(set(my_attr_val_q_gram_dict.keys()) & \
                          set(ot_attr_val_q_gram_dict.keys()))
num_my_attr_val = len(my_attr_val_q_gram_dict)
num_ot_attr_val = len(ot_attr_val_q_gram_dict)

jacc_common_attr_val = float(num_common_attr_val) / \
  (num_my_attr_val + num_ot_attr_val - num_common_attr_val)

print('My database contains %d unique attribute values' % (num_my_attr_val))
print('The other database contains %d unique attribute values' % \
      (num_ot_attr_val))
print('  Number of unique attribute values in common: %d' % \
      (num_common_attr_val))
print('    Jaccard based overlap of common attribute values: %.1f%%' % \
      (100.0*jacc_common_attr_val))
print('')

# -----------------------------------------------------------------------------
# Step 2: Generate Bloom filters from q-gram sets
#
if (num_hash_funct in ['opt', 'opt_half', 'opt_quarter']):

  # Set number of hash functions to have in average 50% of bits set to 1
  # (Linking Sensitive Data book, 2020)
  # num_hash_funct = int(math.ceil(0.5 * BF_LEN / \
  #                                math.floor(avrg_num_q_gram)))
  #
  opt_num_hash_funct = int(round(numpy.log(2.0) * float(bf_len) / \
                                 my_avr_num_q_gram))
  if (num_hash_funct == 'opt'):
    num_hash_funct = opt_num_hash_funct
  elif (num_hash_funct == 'opt_half'):
    num_hash_funct = int(round(opt_num_hash_funct / 2.0))
  elif (num_hash_funct == 'opt_quarter'):
    num_hash_funct = int(round(opt_num_hash_funct / 4.0))
  else:
    raise Exception(num_hash_funct)

my_bf_dict, my_bit_pos_q_gram_dict, my_atom_bf_dict = \
    gen_bloom_filter_dict(my_attr_val_q_gram_dict, hash_type, bf_len,
                          num_hash_funct)


ot_bf_dict, ot_bit_pos_q_gram_dict, ot_atom_bf_dict = \
    gen_bloom_filter_dict(ot_attr_val_q_gram_dict, hash_type, bf_len,
                          num_hash_funct)

#rodando ate aqui

# Header string for results output
#
header = '### command_line_call, encoded_attr_name_str,num_bf_to_attack, ' +\
      'hash_type, num_hash_funct, num_my_attr_val, num_ot_attr_val, ' +\
      'num_common_attr_val, jacc_common_perc, my_num_q_gram, ot_num_q_gram, ' +\
      'num_all_q_gram, num_common_q_gram, bf_seg_len, bf_seg_perc, ' +\
      'my_num_q_gram_bf_seg, my_min_num_bit_pos, my_avr_num_bit_pos, ' +\
      'my_med_num_bit_pos, my_max_num_bit_pos, ot_num_q_gram_bf_seg, ' +\
      'ot_min_num_bit_pos, ot_avr_num_bit_pos, ot_med_num_bit_pos, ' +\
      'ot_max_num_bit_pos, atom_num_corr_1_1_attr_matches, ' +\
      'atom_num_corr_1_m_attr_matches, atom_num_wrong_1_matches, ' +\
      'atom_num_wrong_m_matches, atom_num_no_matches, num_reidentified_ids, ' +\
      'num_reidentified_ids_matches, num_reidentified_ids_no_matches, ' +\
      'reidentified_id_list'

print(header)

if result_file_exists == False:
  f.write(header.split('### ')[1]+'\n')

# -----------------------------------------------------------------------------
# Step 3: Run the segment attack with different segment percentages
#
for bf_seg_perc in bf_seg_perc_list:

  print('-'*80)
  print()

  # Generate a string to be printed as the result summary
  #
  res_str = '### "' + ' '.join(command_line_call)
  res_str = res_str.replace(' , ', ' comma ')  # The separator in input file
  res_str = res_str.replace(',', ';') + '", '

  # Details of how many attribute values in the two data sets, and how many
  # occur in common in both data sets
  #
  res_str += '%s, %s, %s, %d, %d, %d, %d, %.1f, ' % (my_encoded_attr_name_str,
                                                str(num_bf_to_attack),
                                                hash_type,
                                                num_hash_funct, num_my_attr_val,
                                                num_ot_attr_val,
                                                num_common_attr_val,
                                                100.0*jacc_common_attr_val)

  # Details of how many q-grams in the two data sets, how many in total,
  # and how many occur occur in common in both data sets
  #
  res_str += '%d, %d, %d, %d, ' % (my_num_q_gram, ot_num_q_gram, \
                                   num_all_q_gram, num_common_q_gram)

  if hash_type == 'xbf':
    bf_seg_len = int((float(bf_len)/2)*bf_seg_perc/100)
  else:
    bf_seg_len = int(float(bf_len)*bf_seg_perc/100)

  # Get the BF segments from the BFs of both data sets
  #
  my_bf_seg_dict = get_bf_segments(my_bf_dict, bf_seg_len)
  ot_bf_seg_dict = get_bf_segments(ot_bf_dict, bf_seg_len)
  
  my_atom_bf_seg_dict = get_bf_segments(my_atom_bf_dict, bf_seg_len)
  
  my_num_q_gram_bf_seg, my_min_num_bit_pos, my_avr_num_bit_pos, \
         my_med_num_bit_pos, my_max_num_bit_pos = \
                  bf_segment_get_num_q_gram(my_bit_pos_q_gram_dict,
                                            bf_seg_len, num_hash_funct)
  ot_num_q_gram_bf_seg, ot_min_num_bit_pos, ot_avr_num_bit_pos, \
         ot_med_num_bit_pos, ot_max_num_bit_pos = \
                  bf_segment_get_num_q_gram(ot_bit_pos_q_gram_dict,
                                            bf_seg_len, num_hash_funct)

  # Add BF segment information to result string
  #
  res_str += '%d, %d%%, %d, %d, %.2f, %d, %d, ' % (bf_seg_len, bf_seg_perc, \
                                                 my_num_q_gram_bf_seg, \
                                                 my_min_num_bit_pos, \
                                                 my_avr_num_bit_pos, \
                                                 my_med_num_bit_pos, \
                                                 my_max_num_bit_pos)
  res_str += '%d, %d, %.2f, %d, %d, ' % (ot_num_q_gram_bf_seg, \
                                       ot_min_num_bit_pos, \
                                       ot_avr_num_bit_pos, \
                                       ot_med_num_bit_pos, \
                                       ot_max_num_bit_pos)

  # Atom based attack
  #
  num_corr_1_1_attr_matches, num_corr_1_m_attr_matches, \
      num_wrong_1_matches, num_wrong_m_matches, \
      num_no_matches,records_matched = bf_segment_atom_attack(ot_bf_seg_dict,
                                              my_atom_bf_seg_dict,
                                              my_bit_pos_q_gram_dict,
                                              all_q_gram_attr_val_dict,
                                              num_bf_to_attack)

  res_str += '%d, %d, %d, %d, %d, ' % (num_corr_1_1_attr_matches, \
                                     num_corr_1_m_attr_matches, \
                                     num_wrong_1_matches, num_wrong_m_matches, \
                                     num_no_matches)

  # profile 
  #
  num_reidentified_ids,num_reidentified_ids_matches, \
  num_reidentified_ids_no_matches = profile_reidentied_records(DATASET_PATH,
                                          my_data_set_name,ot_data_set_name,
                                          col_sep_char,rec_id_col, attr_list)
  
  res_str += '%d, %d, %d,' % (num_reidentified_ids,num_reidentified_ids_matches,
                              num_reidentified_ids_no_matches)
  
  saida_lista = ''
  for r_id in records_matched:
    saida_lista += str(r_id)+'#'

  res_str += '%s' % (saida_lista) 
  # Print result line for CSV generation
  #

  print(res_str)
  f.write(res_str.split('### ')[1]+'\n') #savinf csv file
  f.flush()

f.close()


In [ ]:
#@title Run attack { form-width: "50%" }

# NUM_MATCHES_TO_TRY = 1000  # or 'all'

# initial parameters
BF_HASH_FUNCT1 = hashlib.sha1
BF_HASH_FUNCT2 = hashlib.md5
random.seed(42)

DATASET_PATH='/tmp/'
try:
  RESULTS_PATH
except NameError:
  RESULTS_PATH='/tmp/'

command_line_call = sys.argv


#@markdown -----------------
#@markdown ##### Dataset:
dataset = "ncvoter-sample-nm-s6" #@param ["ncvoter","ncvoter-sample","ncvoter-sample-nm-s2","ncvoter-sample-nm-s3","ncvoter-sample-nm-s4","ncvoter-sample-nm-s5","ncvoter-sample-nm-s6","politicians","politicians-split-2","politicians-split-3","politicians-split-4","politicians-split-5","politicians-split-6"]

if dataset == 'politicians':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-all-1to1-b.csv.gz'
if dataset == 'politicians-split-2':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i2.csv.gz'
if dataset == 'politicians-split-3':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i3.csv.gz'
if dataset == 'politicians-split-4':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i4.csv.gz'
if dataset == 'politicians-split-5':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i5.csv.gz'
if dataset == 'politicians-split-6':
  my_data_set_name = 'brpoliticians-federal-2014-all-1to1-a.csv.gz'
  ot_data_set_name = 'brpoliticians-federal-2018-5p-i6.csv.gz'

if dataset == 'ncvoter':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-b.csv.gz'
if dataset == 'ncvoter-sample':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b.csv.gz'
if dataset == 'ncvoter-sample-nm-s2':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split2.csv.gz'
if dataset == 'ncvoter-sample-nm-s3':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split3.csv.gz'
if dataset == 'ncvoter-sample-nm-s4':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split4.csv.gz'
if dataset == 'ncvoter-sample-nm-s5':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split5.csv.gz'
if dataset == 'ncvoter-sample-nm-s6':
  my_data_set_name = 'ncvoter-20140619-temporal-balanced-ratio-1to1-a.csv.gz'
  ot_data_set_name = 'sample-ncvoter-20140619-temporal-balanced-ratio-1to1-b-bk_NM-split6.csv.gz'

rec_id_col   = 0   #@param {type:"integer"}

#voter_reg_num,name_prefix,first_name,middle_name,last_name,
#name_suffix,age,gender,race,ethnic,
#street_address,city,state,zip_code,full_phone_num,
#birth_place,register_date,download_month
#@markdown ###### attr_list parameters
#@markdown | ATTRS | POS | ATTRS | POS |
#@markdown |:---:|:---:|:---:|:---:|
#@markdown |<td colspan=2>NCVR <td colspan=2>BR
#@markdown |first_name |3|first_name |1|
#@markdown |first_name , last_name|3,5|first_name , last_name|1,2|
#@markdown | full_name |3,4,5| full_name |3|
#@markdown |first_name , last_name , address|3,5,11|full_name, address|3,9,10|
# <sup> NCVR </sup>
# > <sup>3,5 - first_name , last_name</sup>\
# > <sup>3,5,11 - first_name , last_name, street_address</sup>\
# > <sup>3,5,6,7,12 - first_name , last_name, age, gender, city</sup>
attr_list        =    [3,5]#@param {type:"raw"}
#@markdown ###### col separator [(',' - ncvr),(';' - brpol)]
col_sep_char = ',' #@param {type:"string"}[',',';']
header_line_flag = True #@param {type:"raw"}

#@markdown -----------------
#@markdown #### Anonymization Parameter:

q =  2#@param {type:"integer"}
hash_type = "xrh" #@param ["rh","xrh","xbf","dh"]
num_hash_funct = "opt" #@param ["opt", "opt_half", "opt_quarter"]
bf_len =  200#@param {type:"integer"}

#ajustar depois
bf_seg_perc_list = [100] #@param {type:"raw"}
num_bf_to_attack =  'all'#@param {type:"raw"}

main_start_time = time.time()

# command_line_call = sys.argv
# q =                int(sys.argv[1])
# hash_type =        sys.argv[2].lower()
# num_hash_funct =   sys.argv[3]
# bf_len =           int(sys.argv[4])
# my_data_set_name = sys.argv[5]
# ot_data_set_name = sys.argv[6]
# rec_id_col =       int(sys.argv[7])
# col_sep_char =     sys.argv[8]
# header_line_flag = eval(sys.argv[9])
# attr_list =        eval(sys.argv[10])
# bf_seg_perc_list = eval(sys.argv[11])
# num_bf_to_attack = sys.argv[12]

assert q >= 1, q
assert hash_type in ['dh','rh','xrh','xbf'], hash_type
if num_hash_funct.isdigit():
  num_hash_funct = int(num_hash_funct)
  assert num_hash_funct >= 1, num_hash_funct
else:
  assert num_hash_funct in ['opt', 'opt_half', 'opt_quarter'], num_hash_funct
assert bf_len > 1, bf_len
#
assert rec_id_col >= 0, rec_id_col
assert header_line_flag in [True,False], header_line_flag
assert isinstance(attr_list, list), attr_list
assert isinstance(bf_seg_perc_list, list), bf_seg_perc_list
if (num_bf_to_attack != 'all'):
  num_bf_to_attack = int(num_bf_to_attack)
  assert num_bf_to_attack > 1, num_bf_to_attack

my_base_data_set_name = my_data_set_name.split('/')[-1].replace('.csv', '')
my_base_data_set_name = my_base_data_set_name.replace('.gz','')
ot_base_data_set_name = ot_data_set_name.split('/')[-1].replace('.csv', '')
ot_base_data_set_name = ot_base_data_set_name.replace('.gz','')

assert ',' not in my_base_data_set_name
assert ',' not in ot_base_data_set_name

print('')
print('-'*80)
print('')

ds_name = my_data_set_name.split('-')[0]
res_file = 'real-'+ds_name+'-'+str(num_bf_to_attack)+'-'+str(hash_type)+'-'+str(bf_len)+'.csv'
result_file_exists = os.path.exists(RESULTS_PATH + res_file)

f = open(RESULTS_PATH + res_file, "a")



# -----------------------------------------------------------------------------
# Step 1: Load the data sets and extract q-grams for selected attributes
#
my_attr_val_q_gram_dict, my_q_gram_attr_val_dict, my_avr_num_q_gram, \
   my_all_q_gram_set, my_encoded_attr_name_str = \
                       load_data_set_extract_q_grams(my_data_set_name,
                                                     rec_id_col, attr_list,
                                                     col_sep_char,
                                                     header_line_flag, q)

ot_attr_val_q_gram_dict, ot_q_gram_attr_val_dict, ot_avr_num_q_gram, \
   ot_all_q_gram_set, ot_encoded_attr_name_str = \
                       load_data_set_extract_q_grams(ot_data_set_name,
                                                     rec_id_col, attr_list,
                                                     col_sep_char,
                                                     header_line_flag, q)

assert my_encoded_attr_name_str == ot_encoded_attr_name_str

# Generate a dictionary with all q-grams and their attribute values from both
# data sets
#
my_num_q_gram = len(my_q_gram_attr_val_dict)
ot_num_q_gram = len(ot_q_gram_attr_val_dict)

all_q_gram_attr_val_dict =    {}

#modified to python3
for (q_gram, attr_val_set) in my_q_gram_attr_val_dict.items():
  all_q_gram_attr_val_dict[q_gram] = attr_val_set
for (q_gram, attr_val_set) in ot_q_gram_attr_val_dict.items():
  q_gram_attr_val_set = all_q_gram_attr_val_dict.get(q_gram, set())
  q_gram_attr_val_set = q_gram_attr_val_set | attr_val_set
  all_q_gram_attr_val_dict[q_gram] = q_gram_attr_val_set

num_all_q_gram =    len(all_q_gram_attr_val_dict)
num_common_q_gram = len(set(my_q_gram_attr_val_dict.keys()) & \
                        set(ot_q_gram_attr_val_dict.keys()))

print('A total of %d unique q-grams occur in both data set' % (num_all_q_gram))
print('  Of these, %d occur in both data sets' % (num_common_q_gram))
print('')

num_common_attr_val = len(set(my_attr_val_q_gram_dict.keys()) & \
                          set(ot_attr_val_q_gram_dict.keys()))
num_my_attr_val = len(my_attr_val_q_gram_dict)
num_ot_attr_val = len(ot_attr_val_q_gram_dict)

jacc_common_attr_val = float(num_common_attr_val) / \
  (num_my_attr_val + num_ot_attr_val - num_common_attr_val)

print('My database contains %d unique attribute values' % (num_my_attr_val))
print('The other database contains %d unique attribute values' % \
      (num_ot_attr_val))
print('  Number of unique attribute values in common: %d' % \
      (num_common_attr_val))
print('    Jaccard based overlap of common attribute values: %.1f%%' % \
      (100.0*jacc_common_attr_val))
print('')

# -----------------------------------------------------------------------------
# Step 2: Generate Bloom filters from q-gram sets
#
if (num_hash_funct in ['opt', 'opt_half', 'opt_quarter']):

  # Set number of hash functions to have in average 50% of bits set to 1
  # (Linking Sensitive Data book, 2020)
  # num_hash_funct = int(math.ceil(0.5 * BF_LEN / \
  #                                math.floor(avrg_num_q_gram)))
  #
  opt_num_hash_funct = int(round(numpy.log(2.0) * float(bf_len) / \
                                 my_avr_num_q_gram))
  if (num_hash_funct == 'opt'):
    num_hash_funct = opt_num_hash_funct
  elif (num_hash_funct == 'opt_half'):
    num_hash_funct = int(round(opt_num_hash_funct / 2.0))
  elif (num_hash_funct == 'opt_quarter'):
    num_hash_funct = int(round(opt_num_hash_funct / 4.0))
  else:
    raise Exception(num_hash_funct)

my_bf_dict, my_bit_pos_q_gram_dict, my_atom_bf_dict = \
    gen_bloom_filter_dict(my_attr_val_q_gram_dict, hash_type, bf_len,
                          num_hash_funct)


ot_bf_dict, ot_bit_pos_q_gram_dict, ot_atom_bf_dict = \
    gen_bloom_filter_dict(ot_attr_val_q_gram_dict, hash_type, bf_len,
                          num_hash_funct)

#rodando ate aqui

# Header string for results output
#
header = '### command_line_call, encoded_attr_name_str,num_bf_to_attack, ' +\
      'hash_type, num_hash_funct, num_my_attr_val, num_ot_attr_val, ' +\
      'num_common_attr_val, jacc_common_perc, my_num_q_gram, ot_num_q_gram, ' +\
      'num_all_q_gram, num_common_q_gram, bf_seg_len, bf_seg_perc, ' +\
      'my_num_q_gram_bf_seg, my_min_num_bit_pos, my_avr_num_bit_pos, ' +\
      'my_med_num_bit_pos, my_max_num_bit_pos, ot_num_q_gram_bf_seg, ' +\
      'ot_min_num_bit_pos, ot_avr_num_bit_pos, ot_med_num_bit_pos, ' +\
      'ot_max_num_bit_pos, atom_num_corr_1_1_attr_matches, ' +\
      'atom_num_corr_1_m_attr_matches, atom_num_wrong_1_matches, ' +\
      'atom_num_wrong_m_matches, atom_num_no_matches, num_reidentified_ids, ' +\
      'num_reidentified_ids_matches, num_reidentified_ids_no_matches, ' +\
      'reidentified_id_list'

print(header)

if result_file_exists == False:
  f.write(header.split('### ')[1]+'\n')

# -----------------------------------------------------------------------------
# Step 3: Run the segment attack with different segment percentages
#
for bf_seg_perc in bf_seg_perc_list:

  print('-'*80)
  print()

  # Generate a string to be printed as the result summary
  #
  res_str = '### "' + ' '.join(command_line_call)
  res_str = res_str.replace(' , ', ' comma ')  # The separator in input file
  res_str = res_str.replace(',', ';') + '", '

  # Details of how many attribute values in the two data sets, and how many
  # occur in common in both data sets
  #
  res_str += '%s, %s, %s, %d, %d, %d, %d, %.1f, ' % (my_encoded_attr_name_str,
                                                str(num_bf_to_attack),
                                                hash_type,
                                                num_hash_funct, num_my_attr_val,
                                                num_ot_attr_val,
                                                num_common_attr_val,
                                                100.0*jacc_common_attr_val)

  # Details of how many q-grams in the two data sets, how many in total,
  # and how many occur occur in common in both data sets
  #
  res_str += '%d, %d, %d, %d, ' % (my_num_q_gram, ot_num_q_gram, \
                                   num_all_q_gram, num_common_q_gram)

  if hash_type == 'xbf':
    bf_seg_len = int((float(bf_len)/2)*bf_seg_perc/100)
  else:
    bf_seg_len = int(float(bf_len)*bf_seg_perc/100)

  # Get the BF segments from the BFs of both data sets
  #
  my_bf_seg_dict = get_bf_segments(my_bf_dict, bf_seg_len)
  ot_bf_seg_dict = get_bf_segments(ot_bf_dict, bf_seg_len)
  
  my_atom_bf_seg_dict = get_bf_segments(my_atom_bf_dict, bf_seg_len)
  
  my_num_q_gram_bf_seg, my_min_num_bit_pos, my_avr_num_bit_pos, \
         my_med_num_bit_pos, my_max_num_bit_pos = \
                  bf_segment_get_num_q_gram(my_bit_pos_q_gram_dict,
                                            bf_seg_len, num_hash_funct)
  ot_num_q_gram_bf_seg, ot_min_num_bit_pos, ot_avr_num_bit_pos, \
         ot_med_num_bit_pos, ot_max_num_bit_pos = \
                  bf_segment_get_num_q_gram(ot_bit_pos_q_gram_dict,
                                            bf_seg_len, num_hash_funct)

  # Add BF segment information to result string
  #
  res_str += '%d, %d%%, %d, %d, %.2f, %d, %d, ' % (bf_seg_len, bf_seg_perc, \
                                                 my_num_q_gram_bf_seg, \
                                                 my_min_num_bit_pos, \
                                                 my_avr_num_bit_pos, \
                                                 my_med_num_bit_pos, \
                                                 my_max_num_bit_pos)
  res_str += '%d, %d, %.2f, %d, %d, ' % (ot_num_q_gram_bf_seg, \
                                       ot_min_num_bit_pos, \
                                       ot_avr_num_bit_pos, \
                                       ot_med_num_bit_pos, \
                                       ot_max_num_bit_pos)

  # Atom based attack
  #
  num_corr_1_1_attr_matches, num_corr_1_m_attr_matches, \
      num_wrong_1_matches, num_wrong_m_matches, \
      num_no_matches,records_matched = bf_segment_atom_attack(ot_bf_seg_dict,
                                              my_atom_bf_seg_dict,
                                              my_bit_pos_q_gram_dict,
                                              all_q_gram_attr_val_dict,
                                              num_bf_to_attack)

  res_str += '%d, %d, %d, %d, %d, ' % (num_corr_1_1_attr_matches, \
                                     num_corr_1_m_attr_matches, \
                                     num_wrong_1_matches, num_wrong_m_matches, \
                                     num_no_matches)

  # profile 
  #
  num_reidentified_ids,num_reidentified_ids_matches, \
  num_reidentified_ids_no_matches = profile_reidentied_records(DATASET_PATH,
                                          my_data_set_name,ot_data_set_name,
                                          col_sep_char,rec_id_col, attr_list)
  
  res_str += '%d, %d, %d,' % (num_reidentified_ids,num_reidentified_ids_matches,
                              num_reidentified_ids_no_matches)
  
  saida_lista = ''
  for r_id in records_matched:
    saida_lista += str(r_id)+'#'

  res_str += '%s' % (saida_lista) 
  # Print result line for CSV generation
  #

  print(res_str)
  f.write(res_str.split('### ')[1]+'\n') #savinf csv file
  f.flush()

f.close()
